In [ ]:
from concurrent.futures import ThreadPoolExecutor
import time
from langchain_anthropic import ChatAnthropic
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
import json
import re
# from pygments import highlight
# from pygments.lexers import JsonLexer
# from pygments.formatters import HtmlFormatter
# from IPython.display import HTML
import pandas as pd
# from langchain.chat_models import ChatOpenAI
from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic
from langchain.schema import AIMessage, HumanMessage, SystemMessage
from openai import OpenAI
from typing import List, Tuple, Union, Dict, Any
import ast
from rapidfuzz import fuzz, process
import re
import json
import glob
import os
from config import settings

pd.set_option('display.max_colwidth', 500)

llm_api_key = settings.API_CONFIG.llm_api_key
llm_api_url = settings.API_CONFIG.llm_api_url
client = OpenAI(
    api_key = llm_api_key,
    base_url = llm_api_url
)
# from langchain.chat_models import ChatOpenAI
from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic
from langchain.schema import AIMessage, HumanMessage, SystemMessage

llm_gem = ChatOpenAI(
        temperature=0,
        openai_api_key=llm_api_key,
        openai_api_base=llm_api_url,
        model=settings.ModelConfig.gemma_model,
        max_tokens=settings.ModelConfig.llm_max_tokens,
        seed=42  
        )

llm_ax = ChatOpenAI(
        temperature=0,
        openai_api_key=llm_api_key,
        openai_api_base=llm_api_url,
        model=settings.ModelConfig.ax_model,
        max_tokens=settings.ModelConfig.llm_max_tokens,
         seed=42  
        )

llm_cld = ChatOpenAI(
        temperature=0,
        openai_api_key=llm_api_key,
        openai_api_base=llm_api_url,
        model=settings.ModelConfig.claude_model,
        max_tokens=settings.ModelConfig.llm_max_tokens,
        seed=42  
        )

llm_gen = ChatOpenAI(
        temperature=0,
        openai_api_key=llm_api_key,
        openai_api_base=llm_api_url,
        model=settings.ModelConfig.gemini_model,
        max_tokens=settings.ModelConfig.llm_max_tokens,
        seed=42  
        )

llm_gpt = ChatOpenAI(
        temperature=0,
        openai_api_key=llm_api_key,
        openai_api_base=llm_api_url,
        model=settings.ModelConfig.gpt_model,
        max_tokens=settings.ModelConfig.llm_max_tokens,
        seed=42  
        )

In [ ]:
from typing import List, Tuple, Union, Dict, Any
import ast

import re
import json
import glob

def dataframe_to_markdown_prompt(df, max_rows=None):
    # Limit rows if needed
    if max_rows is not None and len(df) > max_rows:
        display_df = df.head(max_rows)
        truncation_note = f"\n[Note: Only showing first {max_rows} of {len(df)} rows]"
    else:
        display_df = df
        truncation_note = ""
    
    # Convert to markdown
    df_markdown = display_df.to_markdown()
    
    prompt = f"""

    {df_markdown}
    {truncation_note}

    """
    
    return prompt

def replace_strings(text, replacements):
    for old, new in replacements.items():
        text = text.replace(old, new)
        
    return text

def clean_segment(segment):
    """
    Given a segment that is expected to be quoted (i.e. begins and ends with
    the same single or double quote), remove any occurrences of that quote
    from the inner content.
    For example, if segment is:
         "에이닷 T 멤버십 쿠폰함에 "에이닷은통화요약된닷" 입력"
    then the outer quotes are preserved but the inner double quotes are removed.
    """
    segment = segment.strip()
    if len(segment) >= 2 and segment[0] in ['"', "'"] and segment[-1] == segment[0]:
        q = segment[0]
        # Remove inner occurrences of the quote character.
        inner = segment[1:-1].replace(q, '')
        return q + inner + q
    return segment

def split_key_value(text):
    """
    Splits text into key and value based on the first colon that appears
    outside any quoted region.
    If no colon is found outside quotes, the value will be returned empty.
    """
    in_quote = False
    quote_char = ''
    for i, char in enumerate(text):
        if char in ['"', "'"]:
            # Toggle quote state (assumes well-formed starting/ending quotes for each token)
            if in_quote:
                if char == quote_char:
                    in_quote = False
                    quote_char = ''
            else:
                in_quote = True
                quote_char = char
        elif char == ':' and not in_quote:
            return text[:i], text[i+1:]
    return text, ''

def split_outside_quotes(text, delimiter=','):
    """
    Splits the input text on the given delimiter (default comma) but only
    if the delimiter occurs outside of quoted segments.
    Returns a list of parts.
    """
    parts = []
    current = []
    in_quote = False
    quote_char = ''
    for char in text:
        if char in ['"', "'"]:
            # When encountering a quote, toggle our state
            if in_quote:
                if char == quote_char:
                    in_quote = False
                    quote_char = ''
            else:
                in_quote = True
                quote_char = char
            current.append(char)
        elif char == delimiter and not in_quote:
            parts.append(''.join(current).strip())
            current = []
        else:
            current.append(char)
    if current:
        parts.append(''.join(current).strip())
    return parts

def clean_ill_structured_json(text):
    """
    Given a string that is intended to represent a JSON-like structure
    but may be ill-formed (for example, it might contain nested quotes that
    break standard JSON rules), attempt to “clean” it by processing each
    key–value pair.
    
    The function uses the following heuristics:
      1. Split the input text into comma-separated parts (only splitting
         when the comma is not inside a quoted string).
      2. For each part, split on the first colon (that is outside quotes) to separate key and value.
      3. For any segment that begins and ends with a quote, remove any inner occurrences
         of that same quote.
      4. Rejoin the cleaned key and value.
    
    Note: This approach does not build a fully robust JSON parser. For very complex
          or deeply nested ill-structured inputs further refinement would be needed.
    """
    # First, split the text by commas outside of quotes.
    parts = split_outside_quotes(text, delimiter=',')
    
    cleaned_parts = []
    for part in parts:
        # Try to split into key and value on the first colon not inside quotes.
        key, value = split_key_value(part)
        key_clean = clean_segment(key)
        value_clean = clean_segment(value) if value.strip() != "" else ""
        if value_clean:
            cleaned_parts.append(f"{key_clean}: {value_clean}")
        else:
            cleaned_parts.append(key_clean)
    
    # Rejoin the cleaned parts with commas (or you can use another format if desired)
    return ', '.join(cleaned_parts)

def repair_json(broken_json):
    """
    More advanced JSON repair that handles edge cases better
    """
    json_str = broken_json
    
    # Fix unquoted keys
    json_str = re.sub(r'([{,])\s*([a-zA-Z0-9_]+)\s*:', r'\1 "\2":', json_str)
    
    # Fix unquoted values more carefully
    # Split on quotes to avoid modifying content inside strings
    parts = json_str.split('"')
    
    for i in range(0, len(parts), 2):  # Only process parts outside quotes (even indices)
        # Fix unquoted values in this part
        parts[i] = re.sub(r':\s*([a-zA-Z0-9_]+)(?=\s*[,\]\}])', r': "\1"', parts[i])
    
    json_str = '"'.join(parts)
    
    # Fix trailing commas
    json_str = re.sub(r',\s*([}\]])', r'\1', json_str)
    
    return json_str

def extract_json_objects(text):
    # More sophisticated pattern that tries to match proper JSON syntax
    pattern = r'(\{(?:[^{}]|(?:\{(?:[^{}]|(?:\{[^{}]*\}))*\}))*\})'
    
    result = []
    for match in re.finditer(pattern, text):
        potential_json = match.group(0)
        try:
            # Try to parse and validate
            # json_obj = json.loads(repair_json(potential_json))
            json_obj = ast.literal_eval(clean_ill_structured_json(repair_json(potential_json)))
            result.append(json_obj)
        except json.JSONDecodeError:
            # Not valid JSON, skip
            pass
    
    return result

def extract_between(text, start_marker, end_marker):
    start_index = text.find(start_marker)
    if start_index == -1:
        return None
    
    start_index += len(start_marker)
    end_index = text.find(end_marker, start_index)
    if end_index == -1:
        return None
    
    return text[start_index:end_index]

def extract_content(text: str, tag_name: str) -> List[str]:
    pattern = f'<{tag_name}>(.*?)</{tag_name}>'
    matches = re.findall(pattern, text, re.DOTALL)
    return matches

def clean_bad_text(text):
    import re
    
    if not isinstance(text, str):
        return ""
    
    # Remove URLs and emails
    text = re.sub(r'https?://\S+|www\.\S+', ' ', text)
    text = re.sub(r'\S+@\S+', ' ', text)
    
    # Keep Korean, alphanumeric, spaces, and specific punctuation
    text = re.sub(r'[^\uAC00-\uD7A3\u1100-\u11FF\w\s\.\?!,]', ' ', text)
    
    # Normalize whitespace
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

def clean_text(text):
    """
    Cleans text by removing special characters that don't affect fine-tuning.
    Preserves important structural elements like quotes, brackets, and JSON syntax.
    Specifically handles Korean text (Hangul) properly.
    
    Args:
        text (str): The input text to clean
        
    Returns:
        str: Cleaned text ready for fine-tuning
    """
    import re
    
    # Preserve the basic structure by temporarily replacing important characters
    # with placeholder tokens that won't be affected by cleanup
    
    # Step 1: Temporarily replace JSON structural elements
    placeholders = {
        '"': "DQUOTE_TOKEN",
        "'": "SQUOTE_TOKEN",
        "{": "OCURLY_TOKEN",
        "}": "CCURLY_TOKEN",
        "[": "OSQUARE_TOKEN",
        "]": "CSQUARE_TOKEN",
        ":": "COLON_TOKEN",
        ",": "COMMA_TOKEN"
    }
    
    for char, placeholder in placeholders.items():
        text = text.replace(char, placeholder)
    
    # Step 2: Remove problematic characters
    
    # Remove control characters (except newlines, carriage returns, and tabs which can be meaningful)
    text = re.sub(r'[\x00-\x08\x0B\x0C\x0E-\x1F\x7F]', '', text)
    
    # Normalize all types of newlines to \n
    text = re.sub(r'\r\n|\r', '\n', text)
    
    # Remove zero-width characters and other invisible unicode
    text = re.sub(r'[\u200B-\u200D\uFEFF\u00A0]', '', text)
    
    # MODIFIED: Keep Korean characters (Hangul) along with other useful character sets
    # This regex keeps:
    # - ASCII (Basic Latin): \x00-\x7F
    # - Latin-1 Supplement: \u0080-\u00FF
    # - Latin Extended A & B: \u0100-\u017F\u0180-\u024F
    # - Greek and Coptic: \u0370-\u03FF
    # - Cyrillic: \u0400-\u04FF
    # - Korean Hangul Syllables: \uAC00-\uD7A3
    # - Hangul Jamo (Korean alphabet): \u1100-\u11FF
    # - Hangul Jamo Extended-A: \u3130-\u318F
    # - Hangul Jamo Extended-B: \uA960-\uA97F
    # - Hangul Compatibility Jamo: \u3130-\u318F
    # - CJK symbols and punctuation: \u3000-\u303F
    # - Full-width forms (often used with CJK): \uFF00-\uFFEF
    # - CJK Unified Ideographs (Basic common Chinese/Japanese characters): \u4E00-\u9FFF
    
    # Instead of removing characters, we'll define which ones to keep
    allowed_chars_pattern = r'[^\x00-\x7F\u0080-\u00FF\u0100-\u024F\u0370-\u03FF\u0400-\u04FF' + \
                           r'\u1100-\u11FF\u3130-\u318F\uA960-\uA97F\u3000-\u303F' + \
                           r'\uAC00-\uD7A3\uFF00-\uFFEF\u4E00-\u9FFF\n\r\t ]'
    text = re.sub(allowed_chars_pattern, '', text)
    
    # Step 3: Normalize whitespace (but preserve deliberate line breaks)
    text = re.sub(r'[ \t]+', ' ', text)  # Convert multiple spaces/tabs to single space
    
    # First ensure all newlines are standardized
    text = re.sub(r'\r\n|\r', '\n', text)  # Convert all newline variants to \n
    
    # Then normalize multiple blank lines to at most two
    text = re.sub(r'\n\s*\n+', '\n\n', text)  # Convert multiple newlines to at most two
    
    # Step 4: Restore original JSON structural elements
    for char, placeholder in placeholders.items():
        text = text.replace(placeholder, char)
    
    # Step 5: Fix common JSON syntax issues that might remain
    # Fix spaces between quotes and colons in JSON
    text = re.sub(r'"\s+:', r'":', text)
    
    # Fix trailing commas in arrays
    text = re.sub(r',\s*]', r']', text)
    
    # Fix trailing commas in objects
    text = re.sub(r',\s*}', r'}', text)
    
    return text

def remove_control_characters(text):
    if isinstance(text, str):
        # Remove control characters except commonly used whitespace
        return re.sub(r'[\x00-\x08\x0B-\x0C\x0E-\x1F\x7F]', '', text)
    return text

import openai
from langchain.chains import RetrievalQA
from langchain.llms.openai import OpenAIChat  # For compatibility with newer setup

# Create a custom LLM class that uses the OpenAI client directly
class CustomOpenAI:
    def __init__(self, model="skt/gemma3-12b-it"):
        self.model = model
        
    def __call__(self, prompt):
        response = client.chat.completions.create(
            model=self.model,
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.0
        )
        return response.choices[0].message.content

# Create a simple retrieval function
def get_relevant_context(query, vectorstore, topk=5):
    docs = vectorstore.similarity_search(query, k=topk)
    context = "\n\n".join([doc.page_content for doc in docs])
    titles = ", ".join(set([doc.metadata['title'] for doc in docs if 'title' in doc.metadata.keys()]))
    return {'title':titles, 'context':context}

# Create a function to combine everything
def answer_question(query, vectorstore):
    # Get relevant context
    context = get_relevant_context(query, vectorstore)
    
    # Create combined prompt
    prompt = f"Answer the following question based on the provided context:\n\nContext: {context}\n\nQuestion: {query}\n\nAnswer:"
    
    # Use OpenAI directly
    custom_llm = CustomOpenAI()  # Or your preferred model
    response = custom_llm(prompt)
    
    return response

def is_list_of_dicts(var):
    # Check if the variable is a list
    if not isinstance(var, list):
        return False
    
    # Check if the list is not empty and all elements are dictionaries
    if not var:  # Empty list
        return False
        
    # Check that all elements are dictionaries
    return all(isinstance(item, dict) for item in var)

def remove_duplicate_dicts(dict_list):
    result = []
    seen = set()
    for d in dict_list:
        # Convert dictionary to a hashable tuple of items
        t = tuple(sorted(d.items()))
        if t not in seen:
            seen.add(t)
            result.append(d)
    return result

def convert_to_custom_format(json_items):
    custom_format = []
    
    for item in json_items:
        item_name = item.get("item_name_in_message", "")
        item_id = item.get("item_id", "")
        category = item.get("category", "")
        
        # Create custom format for each item
        custom_line = f"[Item Name] {item_name} [Item ID] {item_id} [Item Category] {category}"
        custom_format.append(custom_line)
    
    return "\n".join(custom_format)

def remove_urls(text):
    # Regular expression pattern to match URLs
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    
    # Replace URLs with an empty string
    return url_pattern.sub('', text)

def remove_custom_pattern(text, keyword="바로가기"):
    # Create a pattern that matches any text followed by the specified keyword
    # We escape the keyword to handle any special regex characters it might contain
    escaped_keyword = re.escape(keyword)
    pattern = re.compile(r'.*? ' + escaped_keyword)
    
    # Replace the matched pattern with an empty string
    return pattern.sub('', text)

def select_most_comprehensive(strings):
    """
    Select the most comprehensive string from a list of overlapping strings.
    Returns the longest string that contains other strings as substrings.
    
    Args:
        strings: List of strings to filter
        
    Returns:
        List of most comprehensive strings (usually one, but could be multiple if no containment)
    """
    if not strings:
        return []
    
    # Remove duplicates and sort by length (longest first)
    unique_strings = list(set(strings))
    unique_strings.sort(key=len, reverse=True)
    
    result = []
    
    for current in unique_strings:
        # Check if current string contains any of the strings already in result
        is_contained = any(current in existing for existing in result)
        
        # Check if current string contains other strings not yet in result
        contains_others = any(other in current for other in unique_strings if other != current and other not in result)
        
        # If current is not contained by existing results and either:
        # 1. It contains other strings, or 
        # 2. No strings contain each other (keep all unique)
        if not is_contained:
            # Remove any strings from result that are contained in current
            result = [r for r in result if r not in current]
            result.append(current)
    
    return result

def replace_special_chars_comprehensive(text):
    """
    More comprehensive: Handle various types of special characters.
    """
    # Replace common punctuation with space
    punctuation_pattern = r'[!@#$%^&*()_+\-=\[\]{};\':"\\|,.<>?/~`]'
    text = re.sub(punctuation_pattern, ' ', text)
    
    # Replace other special symbols
    symbol_pattern = r'[₩＄￦※◆▲▼◀▶★☆♪♫♬♩♭♯]'
    text = re.sub(symbol_pattern, ' ', text)
    
    # Replace various dashes and quotes
    dash_quote_pattern = r'[—–‒―""''‚„‹›«»]'
    text = re.sub(dash_quote_pattern, ' ', text)
    
    # Clean up multiple spaces
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

def preprocess_text(text):
    # 특수문자를 공백으로 변환
    text = re.sub(r'[^\w\s]', ' ', text)
    # 여러 공백을 하나로 통일
    text = re.sub(r'\s+', ' ', text)
    # 앞뒤 공백 제거
    return text.strip()

def extract_longest_entities(msg, entity_list):
    """
    msg에서 발견되는 entity_list의 개체명 중 가장 긴 것들만 추출
    (포함 관계가 있는 경우 더 긴 개체명만 선택)
    공백을 제거하고 비교하되, msg에 있는 원본 형태를 반환
    
    Args:
        msg: 검색할 텍스트
        entity_list: 개체명 리스트
    
    Returns:
        msg에 존재하는 개체명 중 다른 개체명에 포함되지 않는 가장 긴 것들의 리스트 (원본 형태)
    """
    import re
    
    # msg에서 공백 제거한 버전
    msg_no_space = msg.replace(' ', '')
    
    # msg에 실제로 존재하는 개체명과 그 위치 찾기
    found_entities = []
    for entity in entity_list:
        entity_no_space = entity.replace(' ', '')
        if entity_no_space in msg_no_space:
            # msg에서 원본 형태 찾기
            start_idx = msg_no_space.find(entity_no_space)
            
            # 공백을 고려하여 원본 텍스트에서 해당 부분 추출
            char_count = 0
            original_start = 0
            original_end = 0
            
            for i, char in enumerate(msg):
                if char != ' ':
                    if char_count == start_idx:
                        original_start = i
                    if char_count == start_idx + len(entity_no_space):
                        original_end = i
                        break
                    char_count += 1
            
            original_entity = msg[original_start:original_end]
            found_entities.append({
                'original': original_entity,
                'no_space': entity_no_space,
                'length': len(entity_no_space)
            })
    
    # 길이 내림차순으로 정렬 (공백 제거 기준)
    found_entities.sort(key=lambda x: x['length'], reverse=True)
    
    # 가장 긴 개체명들만 선택 (공백 제거 버전으로 포함 관계 확인)
    longest_entities = []
    
    for entity_info in found_entities:
        # 현재 entity가 이미 선택된 더 긴 entity에 포함되는지 확인 (공백 제거 버전으로)
        is_contained = False
        for selected_info in longest_entities:
            if entity_info['no_space'] in selected_info['no_space']:
                is_contained = True
                break
        
        # 포함되지 않으면 추가
        if not is_contained:
            longest_entities.append(entity_info)
    
    # 원본 형태만 반환
    return [entity_info['original'].strip() for entity_info in longest_entities]


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from difflib import SequenceMatcher
def advanced_sequential_similarity(str1, str2, metrics=None, visualize=False):
    """
    Calculate multiple character-level similarity metrics between two strings.
    Parameters:
    -----------
    str1 : str
        First string
    str2 : str
        Second string
    metrics : list
        List of metrics to compute. Options:
        ['ngram', 'lcs', 'subsequence', 'difflib']
        If None, all metrics will be computed
    visualize : bool
        If True, visualize the differences between strings
    Returns:
    --------
    dict
        Dictionary containing similarity scores for each metric
    """
    if metrics is None:
        metrics = ['ngram', 'lcs', 'subsequence', 'difflib']
    results = {}
    # Handle empty strings
    if not str1 or not str2:
        return {metric: 0.0 for metric in metrics+['overall']}
    # Prepare strings
    s1, s2 = str1.lower(), str2.lower()
    # 1. N-gram similarity (with multiple window sizes)
    if 'ngram' in metrics:
        ngram_scores = {}
        for window in range(min([len(s1),len(s2),2]), min([5,max([len(s1),len(s2)])+1])):
            # Skip if strings are shorter than window
            if len(s1) < window or len(s2) < window:
                ngram_scores[f'window_{window}'] = 0.0
                continue
            # Generate character n-grams
            ngrams1 = [s1[i:i+window] for i in range(len(s1) - window + 1)]
            ngrams2 = [s2[i:i+window] for i in range(len(s2) - window + 1)]
            # Count matches
            matches = sum(1 for ng in ngrams1 if ng in ngrams2)
            max_possible = max(len(ngrams1), len(ngrams2))
            # Normalize
            score = matches / max_possible if max_possible > 0 else 0.0
            ngram_scores[f'window_{window}'] = score
        # Average of all n-gram scores
        results['ngram'] = max(ngram_scores.values())#sum(ngram_scores.values()) / len(ngram_scores)
        results['ngram_details'] = ngram_scores
    # 2. Longest Common Substring (LCS)
    if 'lcs' in metrics:
        def longest_common_substring(s1, s2):
            # Dynamic programming approach
            m, n = len(s1), len(s2)
            dp = [[0] * (n + 1) for _ in range(m + 1)]
            max_length = 0
            for i in range(1, m + 1):
                for j in range(1, n + 1):
                    if s1[i-1] == s2[j-1]:
                        dp[i][j] = dp[i-1][j-1] + 1
                        max_length = max(max_length, dp[i][j])
            return max_length
        lcs_length = longest_common_substring(s1, s2)
        max_length = max(len(s1), len(s2))
        results['lcs'] = lcs_length / max_length if max_length > 0 else 0.0
    # 3. Longest Common Subsequence
    if 'subsequence' in metrics:
        def longest_common_subsequence(s1, s2):
            # Dynamic programming approach for subsequence
            m, n = len(s1), len(s2)
            dp = [[0] * (n + 1) for _ in range(m + 1)]
            for i in range(1, m + 1):
                for j in range(1, n + 1):
                    if s1[i-1] == s2[j-1]:
                        dp[i][j] = dp[i-1][j-1] + 1
                    else:
                        dp[i][j] = max(dp[i-1][j], dp[i][j-1])
            return dp[m][n]
        subseq_length = longest_common_subsequence(s1, s2)
        max_length = max(len(s1), len(s2))
        results['subsequence'] = subseq_length / max_length if max_length > 0 else 0.0
    # 4. SequenceMatcher from difflib
    if 'difflib' in metrics:
        sm = SequenceMatcher(None, s1, s2)
        results['difflib'] = sm.ratio()
    # Visualization of differences
    if visualize:
        try:
            # Only works in notebooks or environments that support plotting
            sm = SequenceMatcher(None, s1, s2)
            matches = sm.get_matching_blocks()
            # Prepare for visualization
            fig, ax = plt.subplots(figsize=(10, 3))
            # Draw strings as horizontal bars
            ax.barh(0, len(s1), height=0.4, left=0, color='lightgray', alpha=0.3)
            ax.barh(1, len(s2), height=0.4, left=0, color='lightgray', alpha=0.3)
            # Draw matching parts
            for match in matches:
                i, j, size = match
                if size > 0:  # Ignore zero-length matches
                    ax.barh(0, size, height=0.4, left=i, color='green', alpha=0.5)
                    ax.barh(1, size, height=0.4, left=j, color='green', alpha=0.5)
                    # Draw connection lines between matches
                    ax.plot([i + size/2, j + size/2], [0.2, 0.8], 'k-', alpha=0.3)
            # Add string texts
            for i, c in enumerate(s1):
                ax.text(i + 0.5, 0, c, ha='center', va='center')
            for i, c in enumerate(s2):
                ax.text(i + 0.5, 1, c, ha='center', va='center')
            ax.set_yticks([0, 1])
            ax.set_yticklabels(['String 1', 'String 2'])
            ax.set_xlabel('Character Position')
            ax.set_title('Character-Level String Comparison')
            ax.grid(False)
            plt.tight_layout()
            # plt.show()  # Uncomment to display
        except Exception as e:
            print(f"Visualization error: {e}")
    # Calculate overall similarity score (average of all metrics)
    metrics_to_average = [m for m in results.keys() if not m.endswith('_details')]
    results['overall'] = sum(results[m] for m in metrics_to_average) / len(metrics_to_average)
    return results
# advanced_sequential_similarity('시크릿', '시크릿', metrics='ngram')
# advanced_sequential_similarity('에이닷_자사', '에이닷')['overall']

In [ ]:
import difflib
from difflib import SequenceMatcher
import re

def longest_common_subsequence_ratio(s1, s2, normalizaton_value):
    """
    Calculate similarity based on longest common subsequence (LCS).
    Preserves order and gives high scores for substring relationships.
    """
    def lcs_length(x, y):
        m, n = len(x), len(y)
        dp = [[0] * (n + 1) for _ in range(m + 1)]
        
        for i in range(1, m + 1):
            for j in range(1, n + 1):
                if x[i-1] == y[j-1]:
                    dp[i][j] = dp[i-1][j-1] + 1
                else:
                    dp[i][j] = max(dp[i-1][j], dp[i][j-1])
        
        return dp[m][n]
    
    lcs_len = lcs_length(s1, s2)
    if normalizaton_value == 'max':
        max_len = max(len(s1), len(s2))
        return lcs_len / max_len if max_len > 0 else 1.0
    elif normalizaton_value == 'min':
        min_len = min(len(s1), len(s2))
        return lcs_len / min_len if min_len > 0 else 1.0
    elif normalizaton_value == 's1':
        return lcs_len / len(s1) if len(s1) > 0 else 1.0
    elif normalizaton_value == 's2':
        return lcs_len / len(s2) if len(s2) > 0 else 1.0
    else:
        raise ValueError(f"Invalid normalization value: {normalizaton_value}")

# def sequence_matcher_similarity(s1, s2):
#     """
#     Use Python's built-in SequenceMatcher which considers sequence order.
#     """
#     return SequenceMatcher(None, s1, s2).ratio()

def sequence_matcher_similarity(s1, s2, normalizaton_value):
    """Normalize by minimum length (favors shorter strings)"""
    matcher = difflib.SequenceMatcher(None, s1, s2)
    matches = sum(triple.size for triple in matcher.get_matching_blocks())

    normalization_length = min(len(s1), len(s2))
    if normalizaton_value == 'max':
        normalization_length = max(len(s1), len(s2))
    elif normalizaton_value == 's1':
        normalization_length = len(s1)
    elif normalizaton_value == 's2':
        normalization_length = len(s2)
        
    if normalization_length == 0: 
        return 0.0
    
    return matches / normalization_length

def substring_aware_similarity(s1, s2, normalizaton_value):
    """
    Custom similarity that heavily weights substring relationships
    while considering sequence order.
    """
    # Check if one is a substring of the other
    if s1 in s2 or s2 in s1:
        shorter = min(s1, s2, key=len)
        longer = max(s1, s2, key=len)
        # High base score for substring relationship
        base_score = len(shorter) / len(longer)
        # Bonus for exact substring match
        return min(0.95 + base_score * 0.05, 1.0)
    
    # Use LCS ratio for non-substring cases
    return longest_common_subsequence_ratio(s1, s2, normalizaton_value)

def token_sequence_similarity(s1, s2, normalizaton_value, separator_pattern=r'[\s_\-]+'):
    """
    Tokenize strings and calculate similarity based on token sequence overlap.
    Good for product names with separators.
    """
    tokens1 = re.split(separator_pattern, s1.strip())
    tokens2 = re.split(separator_pattern, s2.strip())
    
    # Remove empty tokens
    tokens1 = [t for t in tokens1 if t]
    tokens2 = [t for t in tokens2 if t]
    
    if not tokens1 or not tokens2:
        return 0.0
    
    # Find longest common subsequence of tokens
    def token_lcs_length(t1, t2):
        m, n = len(t1), len(t2)
        dp = [[0] * (n + 1) for _ in range(m + 1)]
        
        for i in range(1, m + 1):
            for j in range(1, n + 1):
                if t1[i-1] == t2[j-1]:
                    dp[i][j] = dp[i-1][j-1] + 1
                else:
                    dp[i][j] = max(dp[i-1][j], dp[i][j-1])
        
        return dp[m][n]
    
    lcs_tokens = token_lcs_length(tokens1, tokens2)
    normalization_tokens = max(len(tokens1), len(tokens2))
    if normalizaton_value == 'min':
        normalization_tokens = min(len(tokens1), len(tokens2))
    elif normalizaton_value == 's1':
        normalization_tokens = len(tokens1)
    elif normalizaton_value == 's2':
        normalization_tokens = len(tokens2)
    
    # print(normalizaton_value, normalization_tokens, lcs_tokens)
        
    return lcs_tokens / normalization_tokens 

def replace_special_chars_with_space(text):
    """
    문자열에서 특수 문자를 공백으로 변환하는 함수
    
    Args:
        text (str): 변환할 문자열
        
    Returns:
        str: 특수 문자가 공백으로 변환된 문자열
    """
    # 영문자, 숫자, 한글을 제외한 모든 문자를 공백으로 변환
    return re.sub(r'[^a-zA-Z0-9가-힣\s]', ' ', text)
 
def combined_sequence_similarity(s1, s2, weights=None, normalizaton_value='max'):
    """
    Combine multiple sequence-aware similarity measures.
    """

    s1 = replace_special_chars_with_space(s1)
    s2 = replace_special_chars_with_space(s2)
    
    if weights is None:
        weights = {
            'substring': 0.1,
            'sequence_matcher': 0.7,
            'token_sequence': 0.2
        }
    
    similarities = {
        'substring': substring_aware_similarity(s1, s2, normalizaton_value),
        'sequence_matcher': sequence_matcher_similarity(s1, s2, normalizaton_value),
        'token_sequence': token_sequence_similarity(s1, s2, normalizaton_value)
    }
    
    combined = sum(similarities[key] * weights[key] for key in weights)
    return combined, similarities


In [ ]:
from joblib import Parallel, delayed

os.environ["TOKENIZERS_PARALLELISM"] = "true"

def fuzzy_similarities(text, entities):
    results = []
    for entity in entities:
        scores = {
            'ratio': fuzz.ratio(text, entity) / 100,
            'partial_ratio': fuzz.partial_ratio(text, entity) / 100,
            'token_sort_ratio': fuzz.token_sort_ratio(text, entity) / 100,
            'token_set_ratio': fuzz.token_set_ratio(text, entity) / 100
        }
        max_score = max(scores.values())
        results.append((entity, max_score))
    return results

def get_fuzzy_similarities(args_dict):
    text = args_dict['text']
    entities = args_dict['entities']
    threshold = args_dict['threshold']
    text_col_nm = args_dict['text_col_nm']
    item_col_nm = args_dict['item_col_nm']

    # Get similarities using auto method selection
    text_processed = preprocess_text(text.lower())
    similarities = fuzzy_similarities(text_processed, entities)
    
    # Filter by threshold and create DataFrame
    filtered_results = [
        {
            text_col_nm: text,
            item_col_nm: entity, 
            "sim": score
        } 
        for entity, score in similarities 
        if score >= threshold
    ]
    
    return filtered_results

def parallel_fuzzy_similarity(texts, entities, threshold=0.5, text_col_nm='sent', item_col_nm='item_nm_alias', n_jobs=None, batch_size=None):
    """
    Batched version for better performance with large datasets.
    """
    if n_jobs is None:
        n_jobs = min(os.cpu_count()-1, 8)  # Limit to 8 jobs max

    if batch_size is None:
        batch_size = max(1, len(entities) // (n_jobs * 2))
        
    # Create batches
    batches = []
    for text in texts:
        for i in range(0, len(entities), batch_size):
            batch = entities[i:i + batch_size]
            batches.append({"text": text, "entities": batch, "threshold": threshold, "text_col_nm": text_col_nm, "item_col_nm": item_col_nm})
    
    # print(f"Processing {len(item_list)} items in {len(batches)|} batches with {n_jobs} jobs...")
    
    # Run parallel jobs
    with Parallel(n_jobs=n_jobs) as parallel:
        batch_results = parallel(delayed(get_fuzzy_similarities)(args) for args in batches)
    
    # # Flatten results
    # similarities = []
    # for batch_result in batch_results:
    #     similarities.extend(batch_result)
    
    return pd.DataFrame(sum(batch_results, []))

def calculate_seq_similarity(args_dict):
    """
    Process a batch of items in one job for better efficiency.
    """
    sent_item_batch = args_dict['sent_item_batch']
    text_col_nm = args_dict['text_col_nm']
    item_col_nm = args_dict['item_col_nm']
    normalizaton_value = args_dict['normalizaton_value']
    
    results = []
    for sent_item in sent_item_batch:
        sent = sent_item[text_col_nm]
        item = sent_item[item_col_nm]
        try:
            sent_processed = preprocess_text(sent.lower())
            item_processed = preprocess_text(item.lower())
            similarity = combined_sequence_similarity(sent_processed, item_processed, normalizaton_value=normalizaton_value)[0]
            results.append({text_col_nm:sent, item_col_nm:item, "sim":similarity})
        except Exception as e:
            print(f"Error processing {item}: {e}")
            results.append({text_col_nm:sent, item_col_nm:item, "sim":0.0})
    
    return results

def parallel_seq_similarity(sent_item_pdf, text_col_nm='sent', item_col_nm='item_nm_alias', n_jobs=None, batch_size=None, normalizaton_value='s2'):
    """
    Batched version for better performance with large datasets.
    """
    if n_jobs is None:
        n_jobs = min(os.cpu_count()-1, 8)  # Limit to 8 jobs max

    if batch_size is None:
        batch_size = max(1, sent_item_pdf.shape[0] // (n_jobs * 2))
        
    # Create batches
    batches = []
    for i in range(0, sent_item_pdf.shape[0], batch_size):
        batch = sent_item_pdf.iloc[i:i + batch_size].to_dict(orient='records')
        batches.append({"sent_item_batch": batch, 'text_col_nm': text_col_nm, 'item_col_nm': item_col_nm, 'normalizaton_value': normalizaton_value})
    
    # print(f"Processing {len(item_list)} items in {len(batches)|} batches with {n_jobs} jobs...")
    
    # Run parallel jobs
    with Parallel(n_jobs=n_jobs) as parallel:
        batch_results = parallel(delayed(calculate_seq_similarity)(args) for args in batches)
    
    # Flatten results
    # similarities = []
    # for batch_result in batch_results:
    #     similarities.extend(batch_result)
    
    return pd.DataFrame(sum(batch_results, []))

In [ ]:
import torch
from datetime import datetime
def save_embeddings_numpy(embeddings, texts, filename):
    """
    Save embeddings as NumPy arrays (.npz format).
    Most common and efficient method.
    """
    if torch.is_tensor(embeddings):
        embeddings = embeddings.cpu().numpy()
    np.savez_compressed(
        filename,
        embeddings=embeddings,
        texts=texts,
        timestamp=str(datetime.now())
    )
    print(f"✅ Saved embeddings to {filename}")
def load_embeddings_numpy(filename):
    """Load embeddings from NumPy .npz file."""
    data = np.load(filename, allow_pickle=True)
    embeddings = data['embeddings']
    texts = data['texts']
    timestamp = data['timestamp'] if 'timestamp' in data else None
    print(f"✅ Loaded {len(embeddings)} embeddings from {filename}")
    if timestamp:
        print(f"   Created: {timestamp}")
    return embeddings, texts


In [ ]:
import os
from sentence_transformers import SentenceTransformer
import torch
def save_sentence_transformer(model_name, save_path):
    """Download and save SentenceTransformer model locally"""
    print(f"Downloading {model_name}...")
    model = SentenceTransformer(model_name)
    # Create directory if it doesn't exist
    os.makedirs(save_path, exist_ok=True)
    # Save the model
    model.save(save_path)
    print(f"Model saved to {save_path}")
    return model
def load_sentence_transformer(model_path, device=None):
    """Load SentenceTransformer model from local path"""
    if device is None:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Loading model from {model_path}...")
    model = SentenceTransformer(model_path).to(device)
    print(f"Model loaded on {device}")
    return model
# Usage
# Save model (do this once)
# model = save_sentence_transformer('jhgan/ko-sbert-nli', './models/ko-sbert-nli')


In [ ]:
from sentence_transformers import SentenceTransformer

def check_mps_availability():
    """Check if MPS is available on this Mac."""
    print(f"PyTorch version: {torch.__version__}")
    print(f"MPS available: {torch.backends.mps.is_available()}")
    print(f"MPS built: {torch.backends.mps.is_built()}")
    
    if torch.backends.mps.is_available():
        print("✅ MPS is available and ready to use!")
        return True
    else:
        print("❌ MPS is not available. Using CPU instead.")
        return False

mps_available = check_mps_availability()
    
# Determine device
if mps_available:
    device = "mps"
elif torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

# emb_model = SentenceTransformer('jhgan/ko-sbert-nli').to(device)
emb_model = load_sentence_transformer('./models/ko-sbert-nli', device)
# emb_model = SentenceTransformer('jhgan/ko-sroberta-multitask').to(device)


In [73]:
item_pdf_raw = pd.read_csv(settings.METADATA_CONFIG.offer_data_path)
item_pdf_raw['ITEM_DESC'] = item_pdf_raw['ITEM_DESC'].astype('str')
item_pdf_raw['ITEM_NM'] = item_pdf_raw.apply(lambda x: x['ITEM_DESC'] if x['ITEM_DMN']=='E' else x['ITEM_NM'], axis=1)

# item_pdf_all = item_pdf_raw.drop_duplicates(['item_nm','item_id'])[['item_nm','item_id','item_desc','item_dmn']].copy()

original_columns = list(item_pdf_raw.columns)
item_pdf_all = item_pdf_raw.rename(columns={c:c.lower() for c in item_pdf_raw.columns})

item_pdf_all['item_ctg'] = None
item_pdf_all['item_emb_vec'] = None
item_pdf_all['ofer_cd'] = item_pdf_all['item_id']
item_pdf_all['oper_dt_hms'] = '20250101000000'

if settings.PROCESSING_CONFIG.excluded_domain_codes_for_items:
    item_pdf_all = item_pdf_all.query("item_dmn not in @settings.PROCESSING_CONFIG.excluded_domain_codes_for_items")

# item_pdf_all.query("rank<1000")[['item_nm']].drop_duplicates().to_csv("./data/item_nm_1000.csv", index=False)
alias_pdf = pd.read_csv(settings.METADATA_CONFIG.alias_rules_path)
alias_list_ext = alias_pdf.query("description=='voca'")[['alias_1','category']].to_dict('records')
for alias in alias_list_ext:
    adf = item_pdf_all.query("item_nm.str.contains(@alias['alias_1']) and item_dmn==@alias['category']")[['item_nm','item_desc','item_dmn']].rename(columns={'item_nm':'alias_2','item_desc':'description','item_dmn':'category'}).drop_duplicates()
    adf['alias_1'] = alias['alias_1']
    adf = adf[alias_pdf.columns]
    alias_pdf = pd.concat([alias_pdf.query(f"alias_1!='{alias['alias_1']}'"), adf])
alia_rule_set = list(zip(alias_pdf['alias_1'], alias_pdf['alias_2']))

def apply_alias_rule(item_nm):
    item_nm_list = [item_nm]

    for r in alia_rule_set:
        if r[0] in item_nm:
            item_nm_list.extend(item_nm.replace(r[0], r[1]).split("&&"))
        if r[1] in item_nm:
            item_nm_list.extend(item_nm.replace(r[1], r[0]).split("&&"))
    return list(set(item_nm_list))

item_pdf_all['item_nm_alias'] = item_pdf_all['item_nm'].apply(apply_alias_rule)

item_pdf_all = item_pdf_all.explode('item_nm_alias')

user_defined_entity = ['AIA Vitality' , '부스트 파크 건대입구' , 'Boost Park 건대입구']
item_pdf_ext = pd.DataFrame([{'item_nm':e,'item_id':e,'item_desc':e, 'item_dmn':'user_defined', 'start_dt':20250101, 'end_dt':99991231, 'rank':1, 'item_nm_alias':e} for e in user_defined_entity])
item_pdf_all = pd.concat([item_pdf_all,item_pdf_ext])

stop_item_names = pd.read_csv(settings.METADATA_CONFIG.stop_items_path)['stop_words'].to_list()

entity_vocab = []
for row in item_pdf_all.to_dict('records'):
    if row['item_nm_alias'] in stop_item_names:
        continue
    entity_vocab.append((row['item_nm_alias'], {'item_nm':row['item_nm'], 'item_id':row['item_id'], 'description':row['item_desc'], 'item_dmn':row['item_dmn'],'item_nm_alias':row['item_nm_alias']}))

entity_list_for_fuzzy = []
for row in item_pdf_all.to_dict('records'):
    entity_list_for_fuzzy.append((row['item_nm_alias'], {'item_nm':row['item_nm'], 'item_id':row['item_id'], 'description':row['item_desc'], 'item_dmn':row['item_dmn'], 'start_dt':row['start_dt'], 'end_dt':row['end_dt'], 'rank':1, 'item_nm_alias':row['item_nm_alias']}))

# text_list_item = [preprocess_text(x).lower() for x in item_pdf_all['item_nm_alias'].tolist()]
# item_embeddings = emb_model.encode(text_list_item
#                             # ,batch_size=64  # Optimal for MPS
#                             ,convert_to_tensor=True
#                             ,show_progress_bar=True)

# save_embeddings_numpy(item_embeddings, text_list_item, './data/item_embeddings_250527.npz')

In [74]:
# item_pdf_raw.query("ITEM_NM.str.contains('IPHONE AIR')")
item_pdf_all.query("item_nm_alias.str.contains('아이폰 에어')")

item_id     item_nm item_dmn   item_desc item_ctg  item_als item_emb_vec  \
7048    A6TT  IPHONE AIR        E  IPHONE AIR     None       NaN         None   
7391    A6VR  IPHONE AIR        E  IPHONE AIR     None       NaN         None   
7403    A6TP  IPHONE AIR        E  IPHONE AIR     None       NaN         None   
7564    A6TX  IPHONE AIR        E  IPHONE AIR     None       NaN         None   
8734    A6V6  IPHONE AIR        E  IPHONE AIR     None       NaN         None   
8797    A6VA  IPHONE AIR        E  IPHONE AIR     None       NaN         None   
8816    A6VF  IPHONE AIR        E  IPHONE AIR     None       NaN         None   

     ofer_cd     oper_dt_hms item_nm_alias  start_dt  end_dt  rank  
7048    A6TT  20250101000000        아이폰 에어       NaN     NaN   NaN  
7391    A6VR  20250101000000        아이폰 에어       NaN     NaN   NaN  
7403    A6TP  20250101000000        아이폰 에어       NaN     NaN   NaN  
7564    A6TX  20250101000000        아이폰 에어       NaN     NaN   NaN  
8734    A6V6  20250101000000        아이폰 에어       NaN     NaN   NaN  
8797    A6VA  20250101000000        아이폰 에어       NaN     NaN   NaN  
8816    A6VF  20250101000000        아이폰 에어       NaN     NaN   NaN

In [ ]:
mms_pdf = pd.read_csv(settings.METADATA_CONFIG.mms_msg_path)
# mms_pdf['msg'] = mms_pdf['msg_nm']+"\n"+mms_pdf['mms_phrs']
# mms_pdf = mms_pdf.groupby(["msg_nm","mms_phrs","msg"])['offer_dt'].min().reset_index(name="offer_dt")
# mms_pdf = mms_pdf.reset_index()
mms_pdf = mms_pdf.astype('str')
# mms_pdf['msg_id'] = mms_pdf.index
# mms_pdf['ext_yn'] = 'N'

# mms_pdf.drop(columns=['ext_yn'], inplace=True)
# mms_pdf.to_csv(settings.METADATA_CONFIG.mms_msg_path, index=False)

# mms_pdf.apply(lambda x: x['msg_nm']+"\t"+x['mms_phrs'].replace("\n", " "), axis=1).to_csv("data/mms_val_251023.csv", index=False)

In [ ]:

import re
num_cand_pgms = 5
pgm_pdf = pd.read_csv(settings.METADATA_CONFIG.pgm_info_path)

clue_embeddings = emb_model.encode(pgm_pdf[["pgm_nm","clue_tag"]].apply(lambda x: preprocess_text(x['pgm_nm'].lower())+" "+x['clue_tag'].lower(), axis=1).tolist()
                            # ,batch_size=64  # Optimal for MPS
                            ,convert_to_tensor=True
                            ,show_progress_bar=True)


In [ ]:
# org_pdf = pd.read_csv(settings.METADATA_CONFIG.org_info_path, encoding='cp949')
# org_pdf['sub_org_cd'] = org_pdf['sub_org_cd'].apply(lambda x: x.zfill(4))

org_pdf = pd.read_csv(settings.METADATA_CONFIG.offer_data_path).query("ITEM_DMN=='R'")
org_pdf = org_pdf.rename(columns={c:c.lower() for c in org_pdf.columns})

# text_list_org_all = org_pdf[["org_abbr_nm","bas_addr","dtl_addr"]].apply(lambda x: preprocess_text(x['org_abbr_nm'].lower())+" "+x['bas_addr'].lower()+" "+x['dtl_addr'].lower(), axis=1).tolist()
# org_all_embeddings = emb_model.encode(text_list_org_all
#                     # ,batch_size=32  # Optimal for MPS
#                     ,convert_to_tensor=True
#                     ,show_progress_bar=True)
# save_embeddings_numpy(org_all_embeddings, text_list_org_all, './data/org_all_embeddings_250605.npz')
# text_list_org_nm = org_pdf[["org_abbr_nm"]].apply(lambda x: preprocess_text(x['org_abbr_nm'].lower()), axis=1).tolist()
# org_nm_embeddings = emb_model.encode(text_list_org_nm
#                     # ,batch_size=32  # Optimal for MPS
#                     ,convert_to_tensor=True
#                     ,show_progress_bar=True)
# save_embeddings_numpy(org_nm_embeddings, text_list_org_nm, './data/org_nm_embeddings_250605.npz')

In [ ]:
org_pdf

In [ ]:
# item_embeddings, text_list_item = load_embeddings_numpy('./data/item_embeddings_250527.npz')
# org_all_embeddings, text_list_org_all = load_embeddings_numpy('./data/org_all_embeddings_250605.npz')
# org_nm_embeddings, text_list_org_nm = load_embeddings_numpy('./data/org_nm_embeddings_250605.npz')
# item_embeddings = torch.from_numpy(item_embeddings).to(device)
# org_all_embeddings = torch.from_numpy(org_all_embeddings).to(device)
# org_nm_embeddings = torch.from_numpy(org_nm_embeddings).to(device)


In [ ]:
def convert_df_to_json_list(df):
    """
    Convert DataFrame to the specific JSON structure you want
    """
    result = []
    # Group by 'item_name_in_msg' to create the main structure
    grouped = df.groupby('item_name_in_msg')
    for item_name_in_msg, group in grouped:
        # Create the main item dictionary
        item_dict = {
            'item_name_in_msg': item_name_in_msg,
            'item_in_voca': []
        }
        # Group by item_nm within each item_name_in_msg to collect item_ids
        item_nm_groups = group.groupby('item_nm')
        for item_nm, item_group in item_nm_groups:
            # Collect all item_ids for this item_nm
            item_ids = list(item_group['item_id'].unique())
            voca_item = {
                'item_nm': item_nm,
                'item_id': item_ids
            }
            item_dict['item_in_voca'].append(voca_item)
        result.append(item_dict)
    return result

<h2 style="color: #006600; font-size: 1.5em;">개채명 추출기 (Kiwi)</h2>

In [ ]:
from kiwipiepy import Kiwi
# Simple approach
kiwi = Kiwi()
entity_list_for_kiwi = list(item_pdf_all['item_nm_alias'].unique())
for w in entity_list_for_kiwi:
    kiwi.add_user_word(w, "NNP")
# for w in stop_item_names:
#     kiwi.add_user_word(w, "NNG")


In [ ]:
def filter_text_by_exc_patterns(sentence, exc_tag_patterns):
    """
    Create a new text by replacing tokens that match exclusion tag patterns with whitespace.
    Handles both individual tags and consecutive tag sequences.
    Preserves original whitespace from the source text.
    """
    
    # Separate individual tags from sequences
    individual_tags = set()
    sequences = []
    
    for pattern in exc_tag_patterns:
        if isinstance(pattern, list):
            if len(pattern) == 1:
                individual_tags.add(pattern[0])
            else:
                sequences.append(pattern)
        else:
            individual_tags.add(pattern)
    
    # Track which tokens to exclude
    tokens_to_exclude = set()
    
    # Check for individual tag matches
    for i, token in enumerate(sentence.tokens):
        if token.tag in individual_tags:
            tokens_to_exclude.add(i)
    
    # Check for sequence matches
    for sequence in sequences:
        seq_len = len(sequence)
        for i in range(len(sentence.tokens) - seq_len + 1):
            # Check if consecutive tokens match the sequence
            if all(sentence.tokens[i + j].tag == sequence[j] for j in range(seq_len)):
                # Mark all tokens in this sequence for exclusion
                for j in range(seq_len):
                    tokens_to_exclude.add(i + j)
    
    # Create a character array from the original text
    result_chars = list(sentence.text)
    
    # Replace excluded tokens with whitespace while preserving original whitespace
    for i, token in enumerate(sentence.tokens):
        if i in tokens_to_exclude:
            # Replace token characters with spaces, but keep original whitespace intact
            start_pos = token.start - sentence.start  # Adjust for sentence start offset
            end_pos = start_pos + token.len
            for j in range(start_pos, end_pos):
                if j < len(result_chars) and result_chars[j] != ' ':
                    result_chars[j] = ' '
    
    # Join the character array to create filtered text
    filtered_text = ''.join(result_chars)

    #Replace consecutive whitespaces with a single whitespace
    filtered_text = re.sub(r'\s+', ' ', filtered_text)
    
    return filtered_text

# Define Token and Sentence classes
class Token:
    def __init__(self, form, tag, start, length):
        self.form = form
        self.tag = tag
        self.start = start
        self.len = length

class Sentence:
    def __init__(self, text, start, end, tokens, subs=None):
        self.text = text
        self.start = start
        self.end = end
        self.tokens = tokens
        self.subs = subs or []

exc_tag_patterns = [['SN', 'NNB'],
 ['W_SERIAL'],
 ['JKO'],
 ['W_URL'],
 ['W_EMAIL'],
 ['XSV', 'EC'],
 ['VV', 'EC'],
 ['VCP', 'ETM'],
 ['XSA', 'ETM'],
 ['VV', 'ETN'],
 ['W_SERIAL'],
 ['W_URL'],
 ['JKO'],
 ['SSO'],
 ['SSC'],
 ['SW'],
 ['SF'],
 ['SP'],
 ['SS'],
 ['SE'],
 ['SO'],
 ['SB'],
 ['SH'],
 ['W_HASHTAG']
 ]

# sentence = sentences[1]

# Apply the filtering
# filtered_text = filter_text_by_exc_patterns(sentence, exc_tag_patterns)

# print("Original text:", repr(sentence.text))
# print("Filtered text:", repr(filtered_text))

# # Show which tokens were excluded
# print("\nToken analysis:")
# individual_tags = set()
# sequences = []

# for pattern in exc_tag_patterns:
#     if isinstance(pattern, list):
#         if len(pattern) == 1:
#             individual_tags.add(pattern[0])
#         else:
#             sequences.append(pattern)
#     else:
#         individual_tags.add(pattern)

# print("Individual exclusion tags:", individual_tags)
# print("Sequence exclusion patterns:", sequences)
# print()

# for i, token in enumerate(sentence.tokens):
#     status = "EXCLUDED" if token.tag in individual_tags else "KEPT"
#     print(f"Token {i}: '{token.form}' ({token.tag}) at pos {token.start}-{token.start + token.len - 1} - {status}")

In [ ]:
msg_text_list = ["""
광고 제목:[SK텔레콤] 2월 0 day 혜택 안내
광고 내용:(광고)[SKT] 2월 0 day 혜택 안내__[2월 10일(토) 혜택]_만 13~34세 고객이라면_베어유 모든 강의 14일 무료 수강 쿠폰 드립니다!_(선착순 3만 명 증정)_▶ 자세히 보기: http://t-mms.kr/t.do?m=#61&s=24589&a=&u=https://bit.ly/3SfBjjc__■ 에이닷 X T 멤버십 시크릿코드 이벤트_에이닷 T 멤버십 쿠폰함에 ‘에이닷이빵쏜닷’을 입력해보세요!_뚜레쥬르 데일리우유식빵 무료 쿠폰을 드립니다._▶ 시크릿코드 입력하러 가기: https://bit.ly/3HCUhLM__■ 문의: SKT 고객센터(1558, 무료)_무료 수신거부 1504
""",
"""
광고 제목:통화 부가서비스를 패키지로 저렴하게!
광고 내용:(광고)[SKT] 콜링플러스 이용 안내  #04 고객님, 안녕하세요. <콜링플러스>에 가입하고 콜키퍼, 컬러링, 통화가능통보플러스까지 총 3가지의 부가서비스를 패키지로 저렴하게 이용해보세요.  ■ 콜링플러스 - 이용요금: 월 1,650원, 부가세 포함 - 콜키퍼(550원), 컬러링(990원), 통화가능통보플러스(770원)를 저렴하게 이용할 수 있는 상품  ■ 콜링플러스 가입 방법 - T월드 앱: 오른쪽 위에 있는 돋보기를 눌러 콜링플러스 검색 > 가입  ▶ 콜링플러스 가입하기: http://t-mms.kr/t.do?m=#61&u=https://skt.sh/17tNH  ■ 유의 사항 - 콜링플러스에 가입하면 기존에 이용 중인 콜키퍼, 컬러링, 통화가능통보플러스 서비스는 자동으로 해지됩니다. - 기존에 구매한 컬러링 음원은 콜링플러스 가입 후에도 계속 이용할 수 있습니다.(시간대, 발신자별 설정 정보는 다시 설정해야 합니다.)  * 최근 다운로드한 음원은 보관함에서 무료로 재설정 가능(다운로드한 날로부터 1년 이내)   ■ 문의: SKT 고객센터(114)  SKT와 함께해주셔서 감사합니다.  무료 수신거부 1504\n    ',
""",
"""
(광고)[SKT] 1월 0 day 혜택 안내_ _[1월 20일(토) 혜택]_만 13~34세 고객이라면 _CU에서 핫바 1,000원에 구매 하세요!_(선착순 1만 명 증정)_▶ 자세히 보기 : http://t-mms.kr/t.do?m=#61&s=24264&a=&u=https://bit.ly/3H2OHSs__■ 에이닷 X T 멤버십 구독캘린더 이벤트_0 day 일정을 에이닷 캘린더에 등록하고 혜택 날짜에 알림을 받아보세요! _알림 설정하면 추첨을 통해 [스타벅스 카페 라떼tall 모바일쿠폰]을 드립니다. _▶ 이벤트 참여하기 : https://bit.ly/3RVSojv_ _■ 문의: SKT 고객센터(1558, 무료)_무료 수신거부 1504
""",
"""
'[T 우주] 넷플릭스와 웨이브를 월 9,900원에! \n(광고)[SKT] 넷플릭스+웨이브 월 9,900원, 이게 되네! __#04 고객님,_넷플릭스와 웨이브 둘 다 보고 싶었지만, 가격 때문에 망설이셨다면 지금이 바로 기회! __오직 T 우주에서만, _2개월 동안 월 9,900원에 넷플릭스와 웨이브를 모두 즐기실 수 있습니다.__8월 31일까지만 드리는 혜택이니, 지금 바로 가입해 보세요! __■ 우주패스 Netflix 런칭 프로모션 _- 기간 : 2024년 8월 31일(토)까지_- 혜택 : 우주패스 Netflix(광고형 스탠다드)를 2개월 동안 월 9,900원에 이용 가능한 쿠폰 제공_▶ 프로모션 자세히 보기: http://t-mms.kr/jAs/#74__■ 우주패스 Netflix(월 12,000원)  _- 기본 혜택 : Netflix 광고형 스탠다드 멤버십_- 추가 혜택 : Wavve 콘텐츠 팩 _* 추가 요금을 내시면 Netflix 스탠다드와 프리미엄 멤버십 상품으로 가입 가능합니다.  __■ 유의 사항_-  프로모션 쿠폰은 1인당 1회 다운로드 가능합니다. _-  쿠폰 할인 기간이 끝나면 정상 이용금액으로 자동 결제 됩니다. __■ 문의: T 우주 고객센터 (1505, 무료)__나만의 구독 유니버스, T 우주 __무료 수신거부 1504'
""",
"""
광고 제목:[SK텔레콤] T건강습관 X AIA Vitality, 우리 가족의 든든한 보험!
광고 내용:(광고)[SKT] 가족의 든든한 보험 (무배당)AIA Vitality 베스트핏 보장보험 안내  고객님, 안녕하세요. 4인 가족 표준생계비, 준비하고 계시나요? (무배당)AIA Vitality 베스트핏 보장보험(디지털 전용)으로 최대 20% 보험료 할인과 가족의 든든한 보험 보장까지 누려 보세요.   ▶ 자세히 보기: http://t-mms.kr/t.do?m=#61&u=https://bit.ly/36oWjgX  ■ AIA Vitality  혜택 - 매달 리워드 최대 12,000원 - 등급 업그레이드 시 특별 리워드 - T건강습관 제휴 할인 최대 40% ※ 제휴사별 할인 조건과 주간 미션 달성 혜택 등 자세한 내용은 AIA Vitality 사이트에서 확인하세요. ※ 이 광고는 AIA생명의 광고이며 SK텔레콤은 모집 행위를 하지 않습니다.  - 보험료 납입 기간 중 피보험자가 장해분류표 중 동일한 재해 또는 재해 이외의 동일한 원인으로 여러 신체 부위의 장해지급률을 더하여 50% 이상인 장해 상태가 된 경우 차회 이후의 보험료 납입 면제 - 사망보험금은 계약일(부활일/효력회복일)로부터 2년 안에 자살한 경우 보장하지 않음 - 일부 특약 갱신 시 보험료 인상 가능 - 기존 계약 해지 후 신계약 체결 시 보험인수 거절, 보험료 인상, 보장 내용 변경 가능 - 해약 환급금(또는 만기 시 보험금이나 사고보험금)에 기타 지급금을 합해 5천만 원까지(본 보험 회사 모든 상품 합산) 예금자 보호 - 계약 체결 전 상품 설명서 및 약관 참조 - 월 보험료 5,500원(부가세 포함)  * 생명보험협회 심의필 제2020-03026호(2020-09-22) COM-2020-09-32426  ■문의: 청약 관련(1600-0880)  무료 수신거부 1504
""",
"""
[SK텔레콤]추석맞이 추가할인 쿠폰 증정
(광고)[SKT]공식인증매장 고촌점 추석맞이 행사__안녕하세요 고객님!_고촌역 1번 출구 고촌파출소 방향 100m SK텔레콤 대리점 입니다._스마트폰 개통, 인터넷/TV 설치 시 조건 없이 추가 할인 행사를 진행합니다.__■삼성 갤럭시 Z플립5/Z폴드5는_  9월 내내 즉시개통 가능!!_1.갤럭시 워치6 개통 시 추가 할인_2.삼성케어+ 파손보장 1년권_3.삼성 정품 악세사리 30% 할인 쿠폰_4.정품 보호필름 1회 무료 부착__■새로운 아이폰15 출시 전_  아이폰14 재고 대방출!!_1.투명 범퍼 케이스 증정_2.방탄 유리 필름 부착_3.25W C타입 PD 충전기__여기에 5만원 추가 할인 적용!!__■기가인터넷+IPTV 가입 시_1.최대 36만원 상당 상품권 지급_2.스마트폰 개통 시 10만원 할인_3.매장 특별 사은품 지급_(특별 사은품은 매장 상황에 따라 변경될 수 있습니다)__■SKT 공식인증매장 고촌점_- 주소: 경기 김포시 고촌읍 장차로 3, SK텔레콤_- 연락처: 0507-1480-7833_- 네이버 예약하기: http://t-mms.kr/bSo/#74_- 매장 홈페이지: http://t-mms.kr/bSt/#74__■ 문의 : SKT 고객센터(1558, 무료)_무료 수신거부 1504_
"""
]
message_idx = 0
mms_msg = msg_text_list[message_idx]


In [ ]:
def filter_specific_terms(strings: List[str]) -> List[str]:
    unique_strings = list(set(strings))  # 중복 제거
    unique_strings.sort(key=len, reverse=True)  # 길이 기준 내림차순 정렬

    filtered = []
    for s in unique_strings:
        if not any(s in other for other in filtered):
            filtered.append(s)

    return filtered

def extract_entities_from_kiwi(mms_msg, item_pdf_all, stop_item_names):
    sentences = sum(kiwi.split_into_sents(re.split(r"_+",mms_msg), return_tokens=True, return_sub_sents=True), [])
    # sentence_list = [sent.text.strip() for sent in sentences if sent.text.strip()]

    sentences_all = []
    for sent in sentences:
        # print(sent.text.strip())
        # print("-"*100)
        if sent.subs:
            for sub_sent in sent.subs:
                sentences_all.append(sub_sent)
        else:
            sentences_all.append(sent)

    sentence_list = []
    for sent in sentences_all:
        # print(sent.text, " --> ", filter_text_by_exc_patterns(sent, exc_tag_patterns))
        sentence_list.append(filter_text_by_exc_patterns(sent, exc_tag_patterns))

    result_msg = kiwi.tokenize(mms_msg, normalize_coda=True, z_coda=False, split_complex=False)
    entities_from_kiwi = []
    for token in result_msg:  # 첫 번째 분석 결과의 토큰 리스트
        if token.tag == 'NNP' and token.form not in stop_item_names+['-'] and len(token.form)>=2 and not token.form.lower() in stop_item_names:  # 고유명사인 경우
        # if token.tag == 'NNG' and token.form in stop_item_names_ext:  # 고유명사인 경우
            entities_from_kiwi.append(token.form)

    from typing import List

    entities_from_kiwi = filter_specific_terms(entities_from_kiwi)

    print("추출된 개체명:", list(set(entities_from_kiwi)))

    similarities_fuzzy = parallel_fuzzy_similarity(
        sentence_list, 
        item_pdf_all['item_nm_alias'].unique(), 
        threshold=0.4,
        text_col_nm='sent',
        item_col_nm='item_nm_alias',
        n_jobs=6,
        batch_size=30
    )

    similarities_seq = parallel_seq_similarity(
        sent_item_pdf=similarities_fuzzy,
        text_col_nm='sent',
        item_col_nm='item_nm_alias',
        n_jobs=6,
        batch_size=100
    )

    cand_items = similarities_seq.query("sim>=0.7 and item_nm_alias.str.contains('', case=False) and item_nm_alias not in @stop_item_names")

    entities_from_kiwi_pdf = item_pdf_all.query("item_nm_alias in @entities_from_kiwi")[['item_nm','item_nm_alias']]
    entities_from_kiwi_pdf['sim'] = 1.0

    cand_item_pdf = pd.concat([cand_items,entities_from_kiwi_pdf])
    cand_item_list = cand_item_pdf.sort_values('sim', ascending=False).groupby(["item_nm_alias"])['sim'].max().reset_index(name='final_sim').sort_values('final_sim', ascending=False).query("final_sim>=0.2")['item_nm_alias'].unique()

    # product_tag = [{"item_name_in_msg":d['item_nm'], "item_in_voca":[{"item_name_in_voca":d['item_nm'], "item_id":d['item_id']}]} for d in item_pdf_all.query("item_nm_alias in @cand_item_list")[['item_nm','item_nm_alias','item_id']].groupby(["item_nm"])['item_id'].apply(list).reset_index().to_dict(orient='records')]

    # product_tag    

    extra_item_pdf = item_pdf_all.query("item_nm_alias in @cand_item_list")[['item_nm','item_nm_alias','item_id']].groupby(["item_nm"])['item_id'].apply(list).reset_index()

    # extra_item_pdf

    return cand_item_list, extra_item_pdf


In [ ]:
def extract_entities_by_logic(cand_entities, threshold_for_fuzzy=0.5):
    similarities_fuzzy = parallel_fuzzy_similarity(
    cand_entities, 
    item_pdf_all['item_nm_alias'].unique(), 
    threshold=threshold_for_fuzzy,
    text_col_nm='item_name_in_msg',
    item_col_nm='item_nm_alias',
    n_jobs=6,
    batch_size=30
    )

    if similarities_fuzzy.shape[0]>0:

        similarities_fuzzy = parallel_seq_similarity(
            sent_item_pdf=similarities_fuzzy,
            text_col_nm='item_name_in_msg',
            item_col_nm='item_nm_alias',
            n_jobs=6,
            batch_size=30,
            normalizaton_value='s1'
        ).rename(columns={'sim':'sim_s1'}).merge(parallel_seq_similarity(
            sent_item_pdf=similarities_fuzzy,
            text_col_nm='item_name_in_msg',
            item_col_nm='item_nm_alias',
            n_jobs=6,
            batch_size=30,
            normalizaton_value='s2'
        ).rename(columns={'sim':'sim_s2'}), on=['item_name_in_msg','item_nm_alias'])
        
        similarities_fuzzy = similarities_fuzzy.query(f"sim_s1>={threshold_for_fuzzy} and sim_s2>={threshold_for_fuzzy}")

        similarities_fuzzy = similarities_fuzzy.groupby(['item_name_in_msg','item_nm_alias'])[['sim_s1','sim_s2']].apply(lambda x: x['sim_s1'].sum() + x['sim_s2'].sum()).reset_index(name='sim')

    return similarities_fuzzy

In [ ]:
cand_entities = ['A.(에이닷)', 'A.알람']

extract_entities_by_logic(cand_entities)#.query("item_nm_alias == '에이닷 전화'")

In [ ]:
# from langchain.prompts import PromptTemplate
# from config.settings import API_CONFIG, MODEL_CONFIG, PROCESSING_CONFIG, METADATA_CONFIG, EMBEDDING_CONFIG
# from prompts import (
#     build_extraction_prompt,
#     enhance_prompt_for_retry,
#     get_fallback_result,
#     build_entity_extraction_prompt,
#     DEFAULT_ENTITY_EXTRACTION_PROMPT,
#     DETAILED_ENTITY_EXTRACTION_PROMPT
# )

# msg_text = msg_text_list[0]
# cand_entities_by_sim = sorted([e.strip() for e in extract_entities_by_logic([msg_text], threshold_for_fuzzy=0.5)['item_nm_alias'].unique() if e.strip() not in stop_item_names and len(e.strip())>=2])

# def get_entities_by_llm(args_dict):

#     llm_model, msg_text = args_dict['llm_model'], args_dict['msg_text']

#     zero_shot_prompt = PromptTemplate(
#     input_variables=["msg","cand_entities"],
#     template="""
#     {entity_extraction_prompt}
    
#     ## message:                
#     {msg}

#     """
#     )
    
#     chain = zero_shot_prompt | llm_model
#     cand_entities = chain.invoke({"entity_extraction_prompt": DETAILED_ENTITY_EXTRACTION_PROMPT, "msg": msg_text}).content
#     cand_entity_list = [e.strip() for e in cand_entities.split(',') if e.strip()]
#     cand_entity_list = [e for e in cand_entity_list if e not in stop_item_names and len(e)>=2]

#     return cand_entity_list


# batches = []
# for llm_model in [llm_cld, llm_ax, llm_gpt]:
#     batches.append({"msg_text": msg_text, "llm_model": llm_model, "cand_entities_list": cand_entities_by_sim})

# # print(f"Processing {len(item_list)} items in {len(batches)|} batches with {n_jobs} jobs...")

# # Run parallel jobs
# with Parallel(n_jobs=3, backend='threading') as parallel:
#     batch_results = parallel(delayed(get_entities_by_llm)(args) for args in batches)

# cand_entity_list = list(set(sum(batch_results, [])))

# parallel_fuzzy_similarity(
#             cand_entity_list, 
#             item_pdf_all['item_nm_alias'].unique(), 
#             threshold=0.6,
#             text_col_nm='item_name_in_msg',
#             item_col_nm='item_nm_alias',
#             n_jobs=6,
#             batch_size=30
#         )

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

def extract_entities_by_llm(llm_model, msg_text, rank_limit=5):
        """
        Extract entities using LLM-based approach.
        """
        from langchain.prompts import PromptTemplate

        # cand_entities_by_sim = sorted([e.strip() for e in extract_entities_by_logic([msg_text], threshold_for_fuzzy=0.5)['item_nm_alias'].unique() if e.strip() not in stop_item_names and len(e.strip())>=2])
        
        zero_shot_prompt = PromptTemplate(
            input_variables=["msg","cand_entities"],
            template="""
            {entity_extraction_prompt}
            
            ## message:                
            {msg}

            """
        )
        # Use the new LangChain pattern instead of deprecated LLMChain
        chain = zero_shot_prompt | llm_model
        cand_entities = chain.invoke({"entity_extraction_prompt": settings.PROCESSING_CONFIG.entity_extraction_prompt, "msg": msg_text}).content

        # Filter out stop words
        cand_entity_list = [e.strip() for e in cand_entities.split(',') if e.strip()]
        cand_entity_list = [e for e in cand_entity_list if e not in stop_item_names and len(e)>=2]

        if not cand_entity_list:
            return pd.DataFrame()

        # Fuzzy similarity matching
        similarities_fuzzy = parallel_fuzzy_similarity(
            cand_entity_list, 
            item_pdf_all['item_nm_alias'].unique(), 
            threshold=0.6,
            text_col_nm='item_name_in_msg',
            item_col_nm='item_nm_alias',
            n_jobs=6,
            batch_size=30
        )
        
        if similarities_fuzzy.empty:
            return pd.DataFrame()
        
        # Filter out stop words from results
        similarities_fuzzy = similarities_fuzzy[~similarities_fuzzy['item_nm_alias'].isin(stop_item_names)]

        # Sequence similarity matching
        cand_entities_sim = parallel_seq_similarity(
            sent_item_pdf=similarities_fuzzy,
            text_col_nm='item_name_in_msg',
            item_col_nm='item_nm_alias',
            n_jobs=6,
            batch_size=30,
            normalizaton_value='s1'
        ).rename(columns={'sim':'sim_s1'}).merge(parallel_seq_similarity(
            sent_item_pdf=similarities_fuzzy,
            text_col_nm='item_name_in_msg',
            item_col_nm='item_nm_alias',
            n_jobs=6,
            batch_size=30,
            normalizaton_value='s2'
        ).rename(columns={'sim':'sim_s2'}), on=['item_name_in_msg','item_nm_alias'])
        
        # Combine similarity scores
        cand_entities_sim = cand_entities_sim.query("sim_s1>=0.5 and sim_s2>=0.5")
        cand_entities_sim = cand_entities_sim.groupby(['item_name_in_msg','item_nm_alias'])[['sim_s1','sim_s2']].apply(lambda x: x['sim_s1'].sum() + x['sim_s2'].sum()).reset_index(name='sim')
        cand_entities_sim = cand_entities_sim.query("sim>=1.1").copy()

        # Rank and limit results
        cand_entities_sim["rank"] = cand_entities_sim.groupby('item_name_in_msg')['sim'].rank(method='first',ascending=False)
        cand_entities_sim = cand_entities_sim.query(f"rank<={rank_limit}").sort_values(['item_name_in_msg','rank'], ascending=[True,True])

        return cand_entities_sim

# extract_entities_by_llm(llm_gem3, msg_text_list[1])


In [ ]:
from langchain.prompts import PromptTemplate
from torch.cuda import temperature
from config.settings import API_CONFIG, MODEL_CONFIG, PROCESSING_CONFIG, METADATA_CONFIG, EMBEDDING_CONFIG
from prompts import (
    build_extraction_prompt,
    enhance_prompt_for_retry,
    get_fallback_result,
    build_entity_extraction_prompt,
    DEFAULT_ENTITY_EXTRACTION_PROMPT,
    DETAILED_ENTITY_EXTRACTION_PROMPT
)

def extract_entities_by_llm_parallel(llm_models, msg_text, rank_limit=200):
        """
        Extract entities using LLM-based approach.
        """
        from langchain.prompts import PromptTemplate

        # cand_entities_by_sim = sorted([e.strip() for e in extract_entities_by_logic([msg_text], threshold_for_fuzzy=0.5)['item_nm_alias'].unique() if e.strip() not in stop_item_names and len(e.strip())>=2])
        
        def get_entities_by_llm(args_dict):

            llm_model, msg_text = args_dict['llm_model'], args_dict['msg_text']

            zero_shot_prompt = PromptTemplate(
            input_variables=["msg","cand_entities"],
            template="""
            {entity_extraction_prompt}
            
            ## message:                
            {msg}

            """
            )
            
            chain = zero_shot_prompt | llm_model
            cand_entities = chain.invoke({"entity_extraction_prompt": DETAILED_ENTITY_EXTRACTION_PROMPT, "msg": msg_text}).content
            cand_entity_list = [e.strip() for e in cand_entities.split(',') if e.strip()]
            cand_entity_list = [e for e in cand_entity_list if e not in stop_item_names and len(e)>=2]

            return cand_entity_list


        batches = []
        for llm_model in llm_models:
            batches.append({"msg_text": msg_text, "llm_model": llm_model})

        # print(f"Processing {len(item_list)} items in {len(batches)|} batches with {n_jobs} jobs...")

        # Run parallel jobs
        with Parallel(n_jobs=3, backend='threading') as parallel:
            batch_results = parallel(delayed(get_entities_by_llm)(args) for args in batches)

        cand_entity_list = list(set(sum(batch_results, [])))

        if not cand_entity_list:
            return pd.DataFrame()

        # Fuzzy similarity matching
        similarities_fuzzy = parallel_fuzzy_similarity(
            cand_entity_list, 
            item_pdf_all['item_nm_alias'].unique(), 
            threshold=0.6,
            text_col_nm='item_name_in_msg',
            item_col_nm='item_nm_alias',
            n_jobs=6,
            batch_size=30
        )

        # print(similarities_fuzzy.query("item_nm_alias.str.contains('프라임')"))
        
        if similarities_fuzzy.empty:
            return pd.DataFrame()
        
        # Filter out stop words from results
        similarities_fuzzy = similarities_fuzzy[~similarities_fuzzy['item_nm_alias'].isin(stop_item_names)]

        # Sequence similarity matching
        try:
            cand_entities_sim = parallel_seq_similarity(
                sent_item_pdf=similarities_fuzzy,
                text_col_nm='item_name_in_msg',
                item_col_nm='item_nm_alias',
                n_jobs=6,
                batch_size=30,
            normalizaton_value='s1'
            ).rename(columns={'sim':'sim_s1'}).merge(parallel_seq_similarity(
                sent_item_pdf=similarities_fuzzy,
                text_col_nm='item_name_in_msg',
                item_col_nm='item_nm_alias',
                n_jobs=6,
                batch_size=30,
                normalizaton_value='s2'
            ).rename(columns={'sim':'sim_s2'}), on=['item_name_in_msg','item_nm_alias'])

            # print(cand_entities_sim.query("item_nm_alias.str.contains('프라임')"))
            
            # Combine similarity scores
            cand_entities_sim = cand_entities_sim.query("sim_s1>=0.4 and sim_s2>=0.4")


            cand_entities_sim = cand_entities_sim.groupby(['item_name_in_msg','item_nm_alias'])[['sim_s1','sim_s2']].apply(lambda x: x['sim_s1'].sum() + x['sim_s2'].sum()).reset_index(name='sim')
            cand_entities_sim = cand_entities_sim.query("sim>=1.1").copy()


            # Rank and limit results
            cand_entities_sim["rank"] = cand_entities_sim.groupby('item_name_in_msg')['sim'].rank(method='first',ascending=False)
            cand_entities_sim = cand_entities_sim.query(f"rank<={rank_limit}").sort_values(['item_name_in_msg','rank'], ascending=[True,True])

            # print(cand_entities_sim['item_nm_alias'].unique())


            zero_shot_prompt = PromptTemplate(
            input_variables=["msg","entities_msg","cand_entities_voca"],
            template="""
            {entity_extraction_prompt}
            
            ## message:                
            {msg}

            ## entities in message:
            {entities_msg}

            ## candidate entities in vocabulary:
            {cand_entities_voca}

            """
            )

            SIMPLE_ENTITY_EXTRACTION_PROMPT = """
            아래 광고 메시지에서 수신자에게 offer하는 개체명을 추출해라. 
            entities in messages는 메시지 내에서 있는 후보 개체명들이다.
            candidate entities in vocabulairy는 entities in messages를 바탕으로 추출한 사전에 있는 후보 개체명들이다.
            메시지와 entities in messages를 참고해서 candidate entities in vocabulary 중에서 유력한 것들을 선택해라.
            
            다음과 같은 포맷으로 결과를 반환해라.
            
            REASON: 선택 이유
            ENTITY: 제공하는 결과는 candidate entities in vocabulary 값들을 ,(콤마)로 연결해라. 없다고 판단하면, 공백으로 결과를 반환해라.
            """
            
                        
            chain = zero_shot_prompt | llm_ax
            cand_entities = chain.invoke({"entity_extraction_prompt": SIMPLE_ENTITY_EXTRACTION_PROMPT, "msg": msg_text, "entities_msg":cand_entities_sim['item_name_in_msg'].unique(), "cand_entities_voca":cand_entities_sim['item_nm_alias'].unique()}).content
            # print(cand_entities)
            cand_entity_list = [e.strip() for e in cand_entities.split("\n")[-1].replace("ENTITY: ","").split(',') if e.strip()]
            cand_entity_list = [e for e in cand_entity_list if e not in stop_item_names and len(e)>=2]

            # print(cand_entity_list)

            cand_entities_sim = cand_entities_sim.query("item_nm_alias in @cand_entity_list")

        except Exception as e:
            print(f"Error in parallel_seq_similarity: {e}")
            return pd.DataFrame()

        return cand_entities_sim

# extract_entities_by_llm(llm_gem3, msg_text_list[1])


In [67]:

msg = """
  message: '"[SK텔레콤] 새샘대리점 역곡점 10월 혜택 안내드립니다.\t(광고)[SKT] 새샘대리점 역곡점 10월 혜택 안내__고객님, 안녕하세요._새샘대리점 역곡점에서 10월 혜택을 안내드립니다._스마트폰 할인과 풍성한 사은품, 신속하고 친절한 서비스까지 모두 경험해 보세요.__■ 10월 특가 스마트폰 기종 안내_- 5GX 프리미엄 요금제 이용 시 갤럭시 Z 플립7(512GB 용량 업그레이드)_- 갤럭시 S25 FE(신제품)_- 갤럭시 퀀텀6__■ 아이폰 17 시리즈 즉시 개통 혜택_- 쓰던 아이폰 반납 시 최대 보상_- 제휴 신용카드로 구매 시 최대 할인_- 선택약정 가입 시 월정액 25% 추가 할인__■ 인터넷+IPTV 가입 혜택_- 기가라이트 인터넷(최대 500Mbps)과 180개 고화질 TV 채널 제공_- 월 이용요금 3만 원대_- 풍성한 사은품 증정__■ 새샘대리점 역곡점_- 주소: 경기도 부천시 원미구 부일로 741_- 연락처: 032-246-5886_- 찾아오시는 길: 역곡역 2번 출구에서 94m 거리 위치_- 영업 시간: 평일 오전 10시~오후 8시, 일요일 오전 10시 30분~오후 7시__▶ 매장 홈페이지 예약/상담: https://t-mms.kr/t.do?m=#61&s=34195&a=&u=http://tworldfriends.co.kr/D153340003__■ 문의: SKT 고객센터(1558, 무료)__SKT와 함께해 주셔서 감사합니다.__무료 수신거부 1504"',
"""


cand_entities_sim = extract_entities_by_llm_parallel([llm_ax], msg, rank_limit=200)

print(cand_entities_sim)



    item_name_in_msg item_nm_alias       sim  rank
50        갤럭시 S25 FE    갤럭시 S25 FE  2.000000   1.0
149        갤럭시 Z 플립7     갤럭시 Z 플립7  2.000000   1.0
174          갤럭시 퀀텀6       갤럭시 퀀텀6  2.000000   1.0
180        기가라이트 인터넷    T_기가라이트인터넷  1.351111   1.0
196       아이폰 17 시리즈        아이폰 17  1.649333   1.0


In [68]:

llm_model_nm = 'ax'

if llm_model_nm == 'ax':
    llm_model = llm_ax
elif llm_model_nm == 'gem':
    llm_model = llm_gem
elif llm_model_nm == 'cld':
    llm_model = llm_cld
elif llm_model_nm == 'gen':
    llm_model = llm_gen
elif llm_model_nm == 'gpt':
    llm_model = llm_gpt

product_info_extraction_mode = 'llm' # options: 'rag', 'llm', 'nlp'
entity_matching_mode = 'llm' # options: 'llm', 'logic'

# for test_text in mms_pdf.query("msg.str.contains('대리점')").sample(10)['msg'].tolist():

test_text = """
  message: '"[SK텔레콤] 새샘대리점 역곡점 10월 혜택 안내드립니다.\t(광고)[SKT] 새샘대리점 역곡점 10월 혜택 안내__고객님, 안녕하세요._새샘대리점 역곡점에서 10월 혜택을 안내드립니다._스마트폰 할인과 풍성한 사은품, 신속하고 친절한 서비스까지 모두 경험해 보세요.__■ 10월 특가 스마트폰 기종 안내_- 5GX 프리미엄 요금제 이용 시 갤럭시 Z 플립7(512GB 용량 업그레이드)_- 갤럭시 S25 FE(신제품)_- 갤럭시 퀀텀6__■ 아이폰 17 시리즈 즉시 개통 혜택_- 쓰던 아이폰 반납 시 최대 보상_- 제휴 신용카드로 구매 시 최대 할인_- 선택약정 가입 시 월정액 25% 추가 할인__■ 인터넷+IPTV 가입 혜택_- 기가라이트 인터넷(최대 500Mbps)과 180개 고화질 TV 채널 제공_- 월 이용요금 3만 원대_- 풍성한 사은품 증정__■ 새샘대리점 역곡점_- 주소: 경기도 부천시 원미구 부일로 741_- 연락처: 032-246-5886_- 찾아오시는 길: 역곡역 2번 출구에서 94m 거리 위치_- 영업 시간: 평일 오전 10시~오후 8시, 일요일 오전 10시 30분~오후 7시__▶ 매장 홈페이지 예약/상담: https://t-mms.kr/t.do?m=#61&s=34195&a=&u=http://tworldfriends.co.kr/D153340003__■ 문의: SKT 고객센터(1558, 무료)__SKT와 함께해 주셔서 감사합니다.__무료 수신거부 1504"',
  
"""


# test_text = msg_text_list[3]

# Revised schema with updated guidelines for preserving original text
schema_prd = {
            "title": "Advertisement title, using the exact expressions as they appear in the original text.",
            "purpose": {
                "type": "array",
                "items": {
                    "type": "string",
                    "enum": ["상품 가입 유도", "대리점/매장 방문 유도", "웹/앱 접속 유도", "이벤트 응모 유도", 
                           "혜택 안내", "쿠폰 제공 안내", "경품 제공 안내", "수신 거부 안내", "기타 정보 제공"]
                },
                "description": "Primary purpose(s) of the advertisement."
            },
            "product": {
                "type": "array",
                "items": {
                    "type": "object",
                    "properties": {
                        "name": {"type": "string", "description": "Name of the advertised product or service."},
                        "action": {
                            "type": "string",
                            "enum": ["구매", "가입", "사용", "방문", "참여", "코드입력", "쿠폰다운로드", "기타"],
                            "description": "Expected customer action for the product."
                        }
                    }
                },
                "description": "Extract all product or service names from the advertisement."
            },
            "channel": {
                "type": "array",
                "items": {
                    "type": "object",
                    "properties": {
                        "type": {
                            "type": "string",
                            "enum": ["URL", "전화번호", "앱", "대리점"],
                            "description": "Channel type."
                        },
                        "value": {"type": "string", "description": "Specific information for the channel. 대리점인 경우 ***점으로 표시될 가능성이 높음"},
                        "action": {
                            "type": "string",
                            "enum": ["가입", "추가 정보", "문의", "수신", "수신 거부"],
                            "description": "Purpose of the channel."
                        }
                    }
                },
                "description": "Channels provided in the advertisement."
            },
            "pgm": {
                "type": "array",
                "items": {"type": "string"},
                "description": "Select the two most relevant pgm_nm from the advertising classification criteria."
            }
        }

print(f"Test text: {test_text.strip()}")
msg = test_text.strip()

cand_item_list, extra_item_pdf = extract_entities_from_kiwi(msg, item_pdf_all, stop_item_names)

product_df = extra_item_pdf.rename(columns={'item_nm':'name'}).query("not name in @stop_item_names")[['name']]
product_df['action'] = '고객에게 기대하는 행동: [구매, 가입, 사용, 방문, 참여, 코드입력, 쿠폰다운로드, 기타] 중에서 선택'
# product_df['position'] = '광고 상품의 분류. [main, sub, etc] 중에서 선택'
product_element = product_df.to_dict(orient='records') if product_df.shape[0]>0 else schema_prd['product']

# print(cand_item_list)

mms_embedding = emb_model.encode([msg.lower()], convert_to_tensor=True)

similarities = torch.nn.functional.cosine_similarity(
    mms_embedding,  
    clue_embeddings,  
    dim=1 
).cpu().numpy()

pgm_pdf_tmp = pgm_pdf.copy()
pgm_pdf_tmp['sim'] = similarities

pgm_pdf_tmp = pgm_pdf_tmp.sort_values('sim', ascending=False)

pgm_cand_info = "\n\t".join(pgm_pdf_tmp.iloc[:num_cand_pgms][['pgm_nm','clue_tag']].apply(lambda x: re.sub(r'\[.*?\]', '', x['pgm_nm'])+" : "+x['clue_tag'], axis=1).to_list())
rag_context = f"\n### 광고 분류 기준 정보 ###\n\t{pgm_cand_info}" if num_cand_pgms > 0 else ""

# 기본 chain of thought (LLM 모드 일관성 강화)
if product_info_extraction_mode == 'llm':
    chain_of_thought = """
1. Identify the advertisement's purpose first, using expressions as they appear in the original text.
2. Extract ONLY explicitly mentioned product/service names from the text, using exact original expressions.
3. For each product, assign a standardized action from: [구매, 가입, 사용, 방문, 참여, 코드입력, 쿠폰다운로드, 기타].
4. Avoid inferring or adding products not directly mentioned in the text.
5. Provide channel information considering the extracted product information, preserving original text expressions.
"""
else:
    chain_of_thought = """
1. Identify the advertisement's purpose first, using expressions as they appear in the original text.
2. Extract product names based on the identified purpose, ensuring only distinct offerings are included and using original text expressions.
3. Provide channel information considering the extracted product information, preserving original text expressions.
"""

prd_ext_guide = """
* Prioritize recall over precision to ensure all relevant products are captured, but verify that each extracted term is a distinct offering.
* Extract all information (title, purpose, product, channel, pgm) using the exact expressions as they appear in the original text without translation, as specified in the schema.
* If the advertisement purpose includes encouraging agency/store visits, provide agency channel information.
"""

if len(cand_item_list) > 0: 
    if product_info_extraction_mode == 'rag':
        rag_context += f"\n\n### 후보 상품 이름 목록 ###\n\t{cand_item_list}"
        prd_ext_guide += f"""
* Use the provided candidate product names as a reference to guide product extraction, ensuring alignment with the advertisement content and using exact expressions from the original text.
        """
    elif product_info_extraction_mode == 'llm':
        # LLM 모드에도 후보 목록 제공하여 일관성 향상
        rag_context += f"\n\n### 참고용 후보 상품 이름 목록 ###\n\t{cand_item_list}"
        prd_ext_guide += f"""
* Refer to the candidate product names list as guidance, but extract products based on your understanding of the advertisement content.
* Maintain consistency by using standardized product naming conventions.
* If multiple similar products exist, choose the most specific and relevant one to reduce variability.
        """
    elif product_info_extraction_mode == 'nlp':
        schema_prd['product'] = product_element  # Assuming product_element is defined elsewhere
        chain_of_thought = """
1. Identify the advertisement’s purpose first, using expressions as they appear in the original text.
2. Extract product information based on the identified purpose, ensuring only distinct offerings are included and using original text expressions.
3. Extract the action field for each product based on the provided name information, derived from the original text context.
4. Provide channel information considering the extracted product information, preserving original text expressions.
        """
        prd_ext_guide += f"""
* Extract the action field for each product based on the identified product names, using the original text context.
        """

schema_prompt = f"""
Return your response as a JSON object that follows this exact structure:

{json.dumps(schema_prd, indent=4, ensure_ascii=False)}

IMPORTANT: 
- Do NOT return the schema definition itself
- Return actual extracted data in the specified format
- For "purpose": return an array of strings from the enum values
- For "product": return an array of objects with "name" and "action" fields
- For "channel": return an array of objects with "type", "value", and "action" fields
- For "pgm": return an array of strings

Example response format:
{{
    "title": "실제 광고 제목",
    "purpose": ["상품 가입 유도", "혜택 안내"],
    "product": [
        {{"name": "실제 상품명", "action": "가입"}},
        {{"name": "다른 상품명", "action": "구매"}}
    ],
    "channel": [
        {{"type": "URL", "value": "실제 URL", "action": "가입"}}
    ],
    "pgm": ["실제 프로그램명"]
}}
"""

# LLM 모드에서 일관성 강화를 위한 추가 지시사항
consistency_note = ""
if product_info_extraction_mode == 'llm':
    consistency_note = """

### 일관성 유지 지침 ###
* 동일한 광고 메시지에 대해서는 항상 동일한 결과를 생성해야 합니다.
* 애매한 표현이 있을 때는 가장 명확하고 구체적인 해석을 선택하세요.
* 상품명은 원문에서 정확히 언급된 표현만 사용하세요.
"""

prompt = f"""
Extract the advertisement purpose and product names from the provided advertisement text.

### Advertisement Message ###
{msg}

### Extraction Steps ###
{chain_of_thought}

### Extraction Guidelines ###
{prd_ext_guide}{consistency_note}

{schema_prompt}

### OUTPUT FORMAT REQUIREMENT ###
You MUST respond with a valid JSON object containing actual extracted data.
Do NOT include schema definitions, type specifications, or template structures.
Return only the concrete extracted information in the specified JSON format.

{rag_context}

### FINAL REMINDER ###
Return a JSON object with actual data, not schema definitions!
"""

# 디버깅을 위한 프롬프트 로깅 (LLM 모드에서만)
if product_info_extraction_mode == 'llm':
    print(f"🔍 LLM 모드 프롬프트 길이: {len(prompt)} 문자")
    print(f"🔍 후보 상품 목록 포함 여부: {'참고용 후보 상품 이름 목록' in rag_context}")

print()

from entity_dag_extractor import DAGParser, extract_dag, create_dag_diagram, sha256_hash, get_root_to_leaf_paths
parser = DAGParser()

extract_entity_dag = False
if extract_entity_dag:
    extract_dag_result = extract_dag(parser, msg, llm_model)

    dag_raw = extract_dag_result['dag_raw']
    dag_section = extract_dag_result['dag_section']
    dag = extract_dag_result['dag']

    create_dag_diagram(dag, filename=f'dag_{sha256_hash(msg)}')
else:
    dag_section = ""

# result_json_text = llm_cld40.invoke(prompt).content
result_json_text = llm_model.invoke(prompt).content

# print(result_json_text)

json_objects_list = extract_json_objects(result_json_text)
if not json_objects_list:
    print("⚠️ LLM이 유효한 JSON 객체를 반환하지 않았습니다")
    print(f"LLM 응답: {result_json_text}")
    json_objects = {
        "title": "광고 메시지",
        "purpose": ["정보 제공"],
        "product": [],
        "channel": [],
        "pgm": []
    }
else:
    json_objects = json_objects_list[-1]
    
    # 스키마 응답 감지
    def is_schema_response(obj):
        """LLM이 스키마 정의를 반환했는지 감지"""
        for field in ['purpose', 'product', 'channel']:
            field_value = obj.get(field, {})
            if isinstance(field_value, dict) and 'type' in field_value and field_value.get('type') == 'array':
                return True
        return False
    
    if is_schema_response(json_objects):
        print("🚨 LLM이 스키마 정의를 반환했습니다! 재시도가 필요합니다.")
        print("현재 응답:", json_objects)
        
        # 강화된 프롬프트로 재시도
        enhanced_prompt = """
🚨 CRITICAL: Return actual extracted data, NOT schema definitions!

DO NOT return: {"purpose": {"type": "array", ...}}
DO return: {"purpose": ["상품 가입 유도"]}

""" + prompt
        
        result_json_text = llm_model.invoke(enhanced_prompt).content
        json_objects_retry = extract_json_objects(result_json_text)
        if json_objects_retry and not is_schema_response(json_objects_retry[-1]):
            json_objects = json_objects_retry[-1]
            print("✅ 재시도 성공: 올바른 데이터 형식 반환")
        else:
            print("❌ 재시도 실패: fallback 결과 사용")

print(json.dumps(json_objects, indent=4, ensure_ascii=False))

if entity_matching_mode == 'logic':
    # 제품 정보 추출 시 스키마 응답 처리
    product_data = json_objects.get('product', [])
    if isinstance(product_data, dict) and 'items' in product_data:
        # 스키마 구조인 경우 빈 리스트 사용
        cand_entities = []
        print("⚠️ product 필드가 스키마 구조입니다. 빈 엔티티 리스트 사용")
    elif isinstance(product_data, list):
        # 올바른 배열 구조
        cand_entities = [item.get('name', '') for item in product_data if isinstance(item, dict) and item.get('name')]
    else:
        cand_entities = []
        print(f"⚠️ 예상하지 못한 product 구조: {type(product_data)}")
    
    # print(cand_entities)
    similarities_fuzzy = extract_entities_by_logic(cand_entities)
elif entity_matching_mode == 'llm':
    similarities_fuzzy = extract_entities_by_llm_parallel([llm_cld], msg)


similarities_fuzzy = similarities_fuzzy[similarities_fuzzy.apply(lambda x: (x['item_nm_alias'].replace(' ', '').lower() in x['item_name_in_msg'].replace(' ', '').lower() or x['item_name_in_msg'].replace(' ', '').lower() in x['item_nm_alias'].replace(' ', '').lower()) , axis=1)]

print(similarities_fuzzy
# .query("item_name_in_msg.str.contains('프라임')")
)

final_result = json_objects.copy()

if num_cand_pgms>0:
    pgm_json = pgm_pdf[pgm_pdf['pgm_nm'].apply(lambda x: re.sub(r'\[.*?\]', '', x) in ' '.join(json_objects['pgm']))][['pgm_nm','pgm_id']].to_dict('records')
    final_result['pgm'] = pgm_json

# print("==="*15+"claude sonnet (emb)"+"==="*15+"\n")
# print(json.dumps(final_result, indent=4, ensure_ascii=False))

print("Entity from extractor:", list(set(cand_item_list)))
print("Entity from LLM:", [x['name'] for x in ([item for item in json_objects['product']['items']] if isinstance(json_objects['product'], dict) else json_objects['product']) ])

if similarities_fuzzy.shape[0]>0:
        # Break down the complex query into simpler steps to avoid pandas/numexpr evaluation error
    # Step 1: Get high similarity items
    high_sim_items = similarities_fuzzy.query('sim >= 1.0')['item_nm_alias'].unique()

    print(high_sim_items)
    
    # Step 2: Filter similarities_fuzzy for conditions
    filtered_similarities = similarities_fuzzy[
        (similarities_fuzzy['item_nm_alias'].isin(high_sim_items)) &
        (~similarities_fuzzy['item_nm_alias'].str.contains('test', case=False)) &
        (~similarities_fuzzy['item_name_in_msg'].isin(stop_item_names))
    ]
    
    # Step 3: Merge with item_pdf_all
    product_tag = convert_df_to_json_list(
        item_pdf_all.merge(filtered_similarities, on=['item_nm_alias'])
    )
    
    # Add action information from original json_objects
    # Create a mapping from item_name_in_msg to action
    action_mapping = {}
    product_data = json_objects.get('product', [])
    if isinstance(product_data, list):
        for item in product_data:
            if isinstance(item, dict) and 'name' in item and 'action' in item:
                action_mapping[item['name']] = item['action']
    
    # Add action to product_tag
    for product in product_tag:
        item_name = product.get('item_name_in_msg', '')
        product['expected_action'] = action_mapping.get(item_name, '기타')

    final_result = {
        "title":json_objects['title'],
        "purpose":json_objects['purpose'],
        "product":product_tag,
        "channel":json_objects['channel'],
        "pgm":json_objects['pgm']
    }

else:
    final_result = json_objects.copy()
    product_tag = [item for item in json_objects['product']['items']] if isinstance(json_objects['product'], dict) else json_objects['product']
    final_result['product'] = [{'item_name_in_msg':d['name'], 'expected_action':d['action'], 'item_in_voca':[{'item_name_in_voca':d['name'], 'item_id': ['#']}]} for d in product_tag if d['name'] not in stop_item_names]
    
if num_cand_pgms>0:
    pgm_json = pgm_pdf[pgm_pdf['pgm_nm'].apply(lambda x: re.sub(r'\[.*?\]', '', x) in ' '.join(json_objects['pgm']))][['pgm_nm','pgm_id']].to_dict('records')
    final_result['pgm'] = pgm_json

channel_tag = []
for d in [item for item in json_objects['channel']['items']] if isinstance(json_objects['channel'], dict) else json_objects['channel']:
    if d['type']=='대리점':

        # _embedding = emb_model.encode([preprocess_text(d['value'].lower())], convert_to_tensor=True)

        # similarities = torch.nn.functional.cosine_similarity(
        #     _embedding,  
        #     org_all_embeddings,  
        #     dim=1 
        # ).cpu().numpy()

        # org_pdf_tmp = org_pdf.copy()
        # org_pdf_tmp['sim'] = similarities.round(5)

        org_pdf_cand = parallel_fuzzy_similarity(
            [preprocess_text(d['value'].lower())], 
            org_pdf['item_nm'].unique(), 
            threshold=0.5,
            text_col_nm='org_nm_in_msg',
            item_col_nm='item_nm',
            n_jobs=6,
            batch_size=100
        ).drop('org_nm_in_msg', axis=1)

        org_pdf_cand = org_pdf.merge(org_pdf_cand, on=['item_nm'])

        org_pdf_cand['sim'] = org_pdf_cand['sim'].round(5)
        
        org_pdf_tmp = org_pdf_cand.sort_values('sim', ascending=False).query("sim>=0.6")
        if org_pdf_tmp.shape[0]<1:
            org_pdf_tmp = org_pdf_cand.sort_values('sim', ascending=False).query("sim>=0.6")

        org_pdf_tmp['sim'] = org_pdf_tmp.apply(lambda x: combined_sequence_similarity(d['value'], x['item_nm'])[0], axis=1)
        org_pdf_tmp['rank'] = org_pdf_tmp['sim'].rank(method='dense',ascending=False)

        org_pdf_tmp = org_pdf_tmp.rename(columns={'item_id':'org_cd','item_nm':'org_nm'})
        org_pdf_tmp = org_pdf_tmp.query("rank==1").groupby('org_nm')['org_cd'].apply(list).reset_index(name='org_cd').to_dict('records')
        # org_nm_id_list =  list(zip(org_pdf_tmp['org_nm'], org_pdf_tmp['org_id']))

        d['store_info'] = org_pdf_tmp
    else:
        d['store_info'] = []

    channel_tag.append(d)

final_result['channel'] = channel_tag
final_result['entity_dag'] = dag_section

# print("==="*15+"claude sonnet (fuzzy)"+"==="*15+"\n")
print(json.dumps(final_result, indent=4, ensure_ascii=False))

print("\n\n")


Test text: message: '"[SK텔레콤] 새샘대리점 역곡점 10월 혜택 안내드립니다.	(광고)[SKT] 새샘대리점 역곡점 10월 혜택 안내__고객님, 안녕하세요._새샘대리점 역곡점에서 10월 혜택을 안내드립니다._스마트폰 할인과 풍성한 사은품, 신속하고 친절한 서비스까지 모두 경험해 보세요.__■ 10월 특가 스마트폰 기종 안내_- 5GX 프리미엄 요금제 이용 시 갤럭시 Z 플립7(512GB 용량 업그레이드)_- 갤럭시 S25 FE(신제품)_- 갤럭시 퀀텀6__■ 아이폰 17 시리즈 즉시 개통 혜택_- 쓰던 아이폰 반납 시 최대 보상_- 제휴 신용카드로 구매 시 최대 할인_- 선택약정 가입 시 월정액 25% 추가 할인__■ 인터넷+IPTV 가입 혜택_- 기가라이트 인터넷(최대 500Mbps)과 180개 고화질 TV 채널 제공_- 월 이용요금 3만 원대_- 풍성한 사은품 증정__■ 새샘대리점 역곡점_- 주소: 경기도 부천시 원미구 부일로 741_- 연락처: 032-246-5886_- 찾아오시는 길: 역곡역 2번 출구에서 94m 거리 위치_- 영업 시간: 평일 오전 10시~오후 8시, 일요일 오전 10시 30분~오후 7시__▶ 매장 홈페이지 예약/상담: https://t-mms.kr/t.do?m=#61&s=34195&a=&u=http://tworldfriends.co.kr/D153340003__■ 문의: SKT 고객센터(1558, 무료)__SKT와 함께해 주셔서 감사합니다.__무료 수신거부 1504"',
추출된 개체명: ['역곡역', '\'"', '갤럭시 S25 FE', '부일로', '경기도', '원미구', '아이폰 17', '부천시', '갤럭시 퀀텀6', '갤럭시 Z 플립7']
🔍 LLM 모드 프롬프트 길이: 7357 문자
🔍 후보 상품 목록 포함 여부: True

{
    "title": "[SK텔레콤] 새샘대리점 역곡점 10월 혜택 안내드립니다.",
    "purpose": [
        "상품 가입 유도",
        "혜택 안내",

In [ ]:
test_text = """
[Web발신]
(광고)[SKT (을지로점)] 신용욱 단골고객님
9월은 SKT 직영점에서 혜택받9, 구매하9

【갤럭시 마지막 특가】
① 와이드 8 ▶기기값 5만원
② A36 ▶기기값 10만원
③ S24 FE ▶기기값 20만원
☞ 제휴카드 사용 시 최대 72만원 추가할인

【SK로 통신사 이동 시】
① 쓰던 폰 그대로 이동시 상품권 20만원
② 인터넷+TV 가입 최대 70만원

★9/9 까지 선착순 행사 (조건에 따라 할인금액 상이)

♥아이폰 17 사전예약♥
고용량 전색상 바로 개통가능☞ https://naver.me/FTM8rdfj

☞ 을지로입구역 5번출구 하나은행 명동사옥 맞은편
https://naver.me/GipIR3Lg
☎ 0507-1399-6011

(무료ARS)수신거부 및 단골해지 : 
080-801-0011
"""
extract_longest_entities(test_text, similarities_fuzzy['item_name_in_msg'].unique())

# similarities_fuzzy['item_name_in_msg'].unique()

In [ ]:
import random
from utils import create_dag_diagram, sha256_hash

llm_model_nm = 'ax'

if llm_model_nm == 'ax':
    llm_model = llm_ax
elif llm_model_nm == 'gem':
    llm_model = llm_gem
elif llm_model_nm == 'cld':
    llm_model = llm_cld
elif llm_model_nm == 'gen':
    llm_model = llm_gen
elif llm_model_nm == 'gpt':
    llm_model = llm_gpt

stop_item_names = pd.read_csv("./data/stop_words.csv")['stop_words'].to_list()

line_break_patterns = {"__":"\n", "■":"\n■", "▶":"\n▶", "_":"\n"}

from entity_dag_extractor import DAGParser, extract_dag, create_dag_diagram, sha256_hash, get_root_to_leaf_paths
parser = DAGParser()

mms_list = [
"""[SK텔레콤] 9월 0 day 혜택 안내
(광고)[SKT] 9월 0 day 혜택 안내
<9월 20일(금) 혜택>
만 13~34세 고객이라면
SKT 0 day
[배달의민족(페리카나 전용) 8,000원 할인]
이게 되네!
(16,000원 이상 주문 시 할인 적용)

 자세히 보기 :  http://t-mms.kr/t.do?m=#61&s=28335&a=&u=https://bit.ly/3XiX8lJ

 에이닷 X T 멤버십 구독캘린더 이벤트
T day 일정을 에이닷 캘린더에 등록하고 혜택 날짜에 알림을 받아보세요!
알림 설정하면 추첨을 통해 [맥도날드 1만원권]을 드립니다.

 이벤트 참여하기 : https://bit.ly/3T95wC2

 문의: SKT 고객센터(1558, 무료)
무료 수신거부 1504"""
,
"""
[SK텔레콤] 아이폰 고객님만을 위한 특별 혜택, 네이버페이 5,000원!
(광고)[SKT]아이폰 고객님만을 위한 특별 혜택, 네이버페이 5,000원!
아이폰을 쓰시는 #04고객님, 안녕하세요?
지금 에이닷에 신규 가입 후 AI 전화 서비스를 이용하시면 네이버페이 포인트 5,000원을 100% 드려요! 선착순 1만명 한정이니 지금 바로 참여해 보세요!

이벤트 참여하기 : http://t-mms.kr/jP6/#74
에이닷 전화 서비스를 이용하시면, 아이폰에서도 AI 통화 녹음/요약 뿐만 아니라 더 강력한 AI 스팸 필터링으로 더 편리하게 통화하실 수 있습니다. 에이닷에서는 모든 AI 서비스가 무료! 지금 바로 경험해 보세요!

 이벤트 관련문의: 더브리즈 02-6101-2000 
SKT와 함께해 주셔서 감사합니다
무료 수신거부 1504
""",
"""[SK텔레콤] 제이스대리점 미사강변점 고객 혜택 안내
(광고)[SKT] 제이스대리점 미사강변점 고객 혜택 안내  고객님, 안녕하세요. SKT 제이스대리점 미사강변점에서 고객님을 위한 혜택을 준비했습니다.   
 갤럭시 S21 구매 고객 혜택(3월 한정)  - 5GX 프라임 요금제 지원금 45만원 100% + 15% 추가 지원금 적용  - 사용하던 기기 반납시 "2배보상"  - 제휴 삼성카드 가입 시 추가 4% 할인  - 제휴 롯데카드 가입 시 5% 할인  - 버즈라이브 50% 할인쿠폰 제공   ※ 모든 혜택조건은 중복 적용가능  
 초등학생 신학기 이벤트  - 가장 많이 찾는 "ZEM폰, 미니폰" 매장 입고되어 즉시개통 가능  - 원격으로 아이 휴대폰 관리부터 실시간 위치 조회 가능  - 기기값 부담은 확 낮추고 결합할인을 통해 더블할인까지~   
 제이스대리점 미사강변점 - 주소: 경기 하남시 미사강변대로64 제일아이조움 1층 T월드 - 연락처: 031-795-2423 ※ 건물 내 무료 주차  
 매장 홈페이지/예약/상담: http://t-mms.kr/t.do?m=#61&s=556&a=&u=https://tworldfriends.co.kr/D145710102 
 매장 위치 보기: http://t-mms.kr/t.do?m=#61&s=557&a=&u=http://kko.to/vtT5qpJDp  
 문의: SKT 고객센터(1558, 무료) ※ 코로나19 확산으로 고객센터에 문의가 증가하고 있습니다. 고객센터와 전화 연결이 원활하지 않을 수 있으니 양해 바랍니다.  SKT와 함께해주셔서 감사합니다. 무료 수신거부 1504"""
 ,
 """수도권 부스트 파크 이벤트를 안내드립니다. 
(광고)[SKT] 부스트 파크 추첨 이벤트 안내  고객님, 안녕하세요. 수도권 부스트 파크에서 진행되는 특별한 이벤트를 안내해드립니다!  
 이벤트1. 갤럭시 노트20 사전예약 이벤트  - 기간: 2020년 8월 7일(금)~8월 13일(목) - 장소: 부스트 파크 강남역/가로수길, 잠실, 광화문, 대학로, 인천 구월, 안양 범계, 일산 라페/웨돔, 수원 광교에 있는 T월드 매장 - 대상: 부스트 파크 T월드 매장에서 5G 체험한 뒤 갤럭시 노트20 사전예약하시는 고객님 - 사은품: 부스트 파크 상생 제휴처 사은품 증정 ※ 사은품은 매장마다 다를 수 있으며, 일찍 소진될 수 있습니다.  
 부스트 파크 위치 보기: http://t-mms.kr/t.do?m=#61&u=https://bit.ly/3iduqLN  
 이벤트2. 갤럭시 노트20 개통 고객 대상 추첨 이벤트  - 기간: 2020년 8월 14일(금)~8월 20일(목) - 대상: 이벤트1에 참여하고 갤럭시 노트20 개통하신 고객님 중 25명 추첨 - 사은품: 인사동 나인트리 프리미어 호텔 1박2일 숙박권 
 이벤트 세부 내용 보기: http://t-mms.kr/t.do?m=#61&u=https://bit.ly/2XS94fl  ※ 갤럭시 노트20 제조사 예약/개통 프로모션과 별개이며, 중복 혜택을 받으실 수 있습니다.  SKT와 함께해주셔서 감사합니다.  무료 수신거부 1504"""
,
"""[SK텔레콤] 엄마손대리점 본점 폴더블6 출시 이벤트 안내드립니다
(광고)[SKT] 엄마손대리점 본점 폴더블6 출시 이벤트
고객님 안녕하세요
SK텔레콤 공식인증대리점 엄마손대리점 본점입니다

■ 폴더블6 
전작 대비 더욱 강력해진 성능으로 19일 첫 출시!
지금 바로 예약(7월 18일 공식예약 마감)하시고 예약고객만의 다양한 혜택을 누려보시기 바랍니다

■ 신형 휴대폰 외 
효도폰, 키즈폰 등 다양한 할인 이벤트도 진행중입니다

■ 추가할인, 사은품 등 다양한 혜택과
요금할인, 숨어있는 포인트 활용 등 꿀Tip상담을 약속드립니다

■ SK공식인증대리점 엄마손대리점 본점
- 주소 : 서울 송파구 가락로 110 
- 연락처 : 02-420-9011
- 구술약도 : 송파역 석촌시장에서 대로변에 위치

▶ 홈페이지 : http://t-mms.kr/t.do?m=#61&s=27256&a=&u=https://naver.me/FyeuZoSp

■ 문의 : SKT 고객센터(1558, 무료)
SK텔레콤과 함께해 주셔서 감사합니다.
무료 수신거부 1504""",

"""[SK텔레콤] 동진대리점 가양역 본점 이벤트 안내
(광고)[SKT] 동진대리점 가양역 본점 이벤트 안내
안녕하세요 고객님
SK텔레콤 가양역 본점에서 검은 토끼의 해 계묘년 특별행사 안내드립니다. 

■ 이 달의 특가 휴대폰!
- 준프리미엄 모델 갤럭시 퀀텀3 최대지원! + 갤럭시 워치까지 무료!(5GX프라임 요금제 이용 조건)
- 자녀분들을 위한 ZEM포켓몬 에디션 이벤트!

■ 초고속 인터넷 + TV 신규가입 또는 약정 만료고객 재약정시 사은품 최대 지급

■ ADT 캡스 홈보안 6개월 무료 체험 (체험 만료시 위약금 면제)

■ 새롭게 출시되는 갤럭시 S23 사전예약 시작
- 2월13일까지 사전예약하면 2월14일에 바로 받아보실 수 있습니다!
매장으로 방문하시면 자세히 설명드리겠습니다.

■ 동진대리점 가양역본점 
- 위치: 서울 강서구 화곡로68길 3 영스퀘어 1층 SK텔레콤
- 연락처: 02-2659-7577

▶카톡:http://pf.kakao.com/
xjwmxaxj

▶바로가기 http://t-mms.kr/t.do?m=#61&s=17996&a=&u=http://t-mms.kr/t.do?m=945711&s=17634&a=&u=http://tworldfriends.co.kr/D151510021
SKT와 함께해주셔서 감사합니다. 
무료 수신거부 1504""",

"""
광고 제목:[SK텔레콤] 2월 0 day 혜택 안내
광고 내용:(광고)[SKT] 2월 0 day 혜택 안내__[2월 10일(토) 혜택]_만 13~34세 고객이라면_베어유 모든 강의 14일 무료 수강 쿠폰 드립니다!_(선착순 3만 명 증정)_▶ 자세히 보기: http://t-mms.kr/t.do?m=#61&s=24589&a=&u=https://bit.ly/3SfBjjc__■ 에이닷 X T 멤버십 시크릿코드 이벤트_에이닷 T 멤버십 쿠폰함에 ‘에이닷이빵쏜닷’을 입력해보세요!_뚜레쥬르 데일리우유식빵 무료 쿠폰을 드립니다._▶ 시크릿코드 입력하러 가기: https://bit.ly/3HCUhLM__■ 문의: SKT 고객센터(1558, 무료)_무료 수신거부 1504

""",

"""
[T 우주] 넷플릭스와 웨이브를 월 9,900원에! \n(광고)[SKT] 넷플릭스+웨이브 월 9,900원, 이게 되네! __#04 고객님,_넷플릭스와 웨이브 둘 다 보고 싶었지만, 가격 때문에 망설이셨다면 지금이 바로 기회! __오직 T 우주에서만, _2개월 동안 월 9,900원에 넷플릭스와 웨이브를 모두 즐기실 수 있습니다.__8월 31일까지만 드리는 혜택이니, 지금 바로 가입해 보세요! __■ 우주패스 Netflix 런칭 프로모션 _- 기간 : 2024년 8월 31일(토)까지_- 혜택 : 우주패스 Netflix(광고형 스탠다드)를 2개월 동안 월 9,900원에 이용 가능한 쿠폰 제공_▶ 프로모션 자세히 보기: http://t-mms.kr/jAs/#74__■ 우주패스 Netflix(월 12,000원)  _- 기본 혜택 : Netflix 광고형 스탠다드 멤버십_- 추가 혜택 : Wavve 콘텐츠 팩 _* 추가 요금을 내시면 Netflix 스탠다드와 프리미엄 멤버십 상품으로 가입 가능합니다.  __■ 유의 사항_-  프로모션 쿠폰은 1인당 1회 다운로드 가능합니다. _-  쿠폰 할인 기간이 끝나면 정상 이용금액으로 자동 결제 됩니다. __■ 문의: T 우주 고객센터 (1505, 무료)__나만의 구독 유니버스, T 우주 __무료 수신거부 1504
"""

]

# for msg in random.sample(mms_pdf.query("msg.str.contains('')")['msg'].unique().tolist(), 1):
for msg in mms_list[-1:]:

    for pattern, replacement in line_break_patterns.items():
        msg = msg.replace(pattern, replacement)

    cand_entities = sorted([e.strip() for e in extract_entities_by_logic([msg], threshold_for_fuzzy=0.7)['item_nm_alias'].unique() if e.strip() not in stop_item_names and len(e.strip())>=2])


    prompt = f"""
## 작업 목표
통신사 광고 메시지에서 **핵심 행동 흐름**을 추출하여 간결한 DAG(Directed Acyclic Graph) 형식으로 표현

## 핵심 원칙
1. **최소 경로 원칙**: 동일한 결과를 얻는 가장 짧은 경로만 표현
2. **핵심 흐름 우선**: 부가적 설명보다 주요 행동 연쇄에 집중
3. **중복 제거**: 의미가 겹치는 노드는 하나로 통합

## 출력 형식
```
(개체명:기대행동) -[관계동사]-> (개체명:기대행동)
또는
(개체명:기대행동)
```

## 개체명 카테고리
### 필수 추출 대상
- **제품/서비스**: 구체적 제품명, 서비스명, 요금제명
- **핵심 혜택**: 금전적 혜택, 사은품, 할인
- **행동 장소**: 온/오프라인 채널 (필요 시만)

### 추출 제외 대상
- 광고 대상자 (예: "아이폰 고객님")
- 일정/기간 정보
- 부가 설명 기능 (핵심 흐름과 무관한 경우)
- 법적 고지사항

## 기대 행동 (10개 표준 동사)
`구매, 가입, 사용, 방문, 참여, 등록, 다운로드, 확인, 수령, 적립`

## 관계 동사 우선순위
### 1순위: 조건부 관계
- `가입하면`, `구매하면`, `사용하면`
- `가입후`, `구매후`, `사용후`

### 2순위: 결과 관계
- `받다`, `수령하다`, `적립하다`

### 3순위: 경로 관계 (필요 시만)
- `통해`, `이용하여`

## DAG 구성 전략
### Step 1: 모든 가치 제안 식별
광고에서 제시하는 **모든 독립적 가치**를 파악
- **즉각적 혜택**: 금전적 보상, 사은품 등
- **서비스 가치**: 제품/서비스 자체의 기능과 혜택
예: "네이버페이 5000원" + "AI 통화 기능 무료 이용"

### Step 2: 독립 경로 구성
각 가치 제안별로 별도 경로 생성:
1. **혜택 획득 경로**: 가입 → 사용 → 보상
2. **서비스 체험 경로**: 가입 → 경험 → 기능 활용

### Step 3: 세부 기능 표현
주요 기능들이 명시된 경우 분기 구조로 표현:
- 통합 가능한 기능은 하나로 (예: AI통화녹음/요약 → "AI통화기능")
- 독립적 기능은 별도로 (예: AI스팸필터링)

## 분석 프로세스 (필수 단계)
### Step 1: 메시지 이해
- 전체 메시지를 한 문단으로 요약
- 광고주의 의도 파악
- **암시된 행동 식별**: 명시되지 않았지만 필수적인 행동 (예: 매장 방문)

### Step 2: 가치 제안 식별
- 즉각적 혜택 (금전, 사은품 등)
- 서비스 가치 (기능, 편의성 등)
- 부가 혜택 (있다면)

### Step 3: Root Node 결정
- **사용자의 첫 번째 행동은 무엇인가?**
- 매장 주소/연락처가 있다면 → 방문이 시작점
- 온라인 링크가 있다면 → 접속이 시작점
- 앱 관련 내용이라면 → 다운로드가 시작점

### Step 4: 관계 분석
- Root Node부터 시작하는 전체 흐름
- 각 행동 간 인과관계 검증
- 조건부 관계 명확화
- 시간적 순서 확인

### Step 5: DAG 구성
- 위 분석을 바탕으로 노드와 엣지 결정
- 중복 제거 및 통합

### Step 6: 자기 검증 및 수정
- **평가**: 초기 DAG를 아래 기준에 따라 검토
  - Root Node가 명확히 식별되었는가? (방문/접속/다운로드 등)
  - 모든 독립적 가치 제안이 포함되었는가? (즉각적 혜택과 서비스 가치)
  - 주요 기능들이 적절히 그룹화되었는가? (중복 제거 여부)
  - 각 경로가 명확한 가치를 전달하는가?
  - 전체 구조가 간결하고 이해하기 쉬운가?
  - 관계 동사가 우선순위에 맞게 사용되었는가? (조건부 > 결과 > 경로)
- **문제 식별**: 위 기준 중 충족되지 않은 항목을 명시하고, 그 이유를 설명
- **수정**: 식별된 문제를 해결한 수정된 DAG를 생성

### Step 7: 최종 검증
- 수정된 DAG가 모든 기준을 충족하는지 재확인
- 만약 문제가 남아있다면, 추가 수정 수행 (최대 2회 반복)
- 최종적으로 모든 기준이 충족되었음을 확인

## 출력 형식 (반드시 모든 섹션 포함)
### 1. 메시지 분석
```
[메시지 요약 및 핵심 의도]
[식별된 가치 제안 목록]
```

### 2. 초기 DAG
```
[초기 DAG 구조]
```

### 3. 자기 검증 결과
```
[평가 기준별 검토 결과]
[식별된 문제점 및 이유]
```

### 4. 수정된 DAG
```
[수정된 DAG 구조]
```

### 5. 최종 검증 및 추출 근거
```
[최종 DAG가 모든 기준을 충족하는 이유]
[노드/엣지 선택의 논리적 근거]
```

## 실행 지침
1. 위 7단계 분석 프로세스를 **순서대로** 수행
2. 각 단계에서 발견한 내용을 **명시적으로** 기록
3. DAG 구성 전 **충분한 분석** 수행
4. 초기 DAG 생성 후 **반드시 자기 검증 및 수정** 수행
5. 최종 출력에 **모든 섹션** 포함
6. **중요**: 분석 과정을 생략하지 말고, 사고 과정과 수정 이유를 투명하게 보여주세요

## 예시 분석
### 잘못된 예시 (핵심 흐름만 추출)
```
(에이닷:가입) -[가입후]-> (AI전화서비스:사용)
(AI전화서비스:사용) -[사용하면]-> (네이버페이5000원:수령)
```
→ 문제: 서비스 자체의 가치(AI 기능들)가 누락됨

### 올바른 예시 (완전한 가치 표현 - Root Node 포함)
```
# 매장 방문부터 시작하는 경로
(제이스대리점:방문) -[방문하여]-> (갤럭시S21:구매)
(갤럭시S21:구매) -[구매시]-> (5GX프라임요금제:가입)
(5GX프라임요금제:가입) -[가입하면]-> (지원금45만원+15%:수령)

# 온라인 시작 경로 예시
(T다이렉트샵:접속) -[접속하여]-> (갤럭시S24:구매)
(갤럭시S24:구매) -[구매시]-> (사은품:수령)
```
→ 장점: 사용자의 첫 행동(Root Node)부터 명확히 표현

## message:
{msg}
"""
    
    print("==="*15+" Message "+"==="*15)
    print(msg)
    # print("==="*15+" DAG (1) "+"==="*15)
    # dag = llm_cld.invoke(prompt_1).content
    # print(dag)

    print("==="*15+f" DAG ({llm_model_nm.upper()}) "+"==="*15)
    dag_raw = llm_model.invoke(prompt).content
    print(dag_raw)

    # NetworkX 그래프로 활용
    dag_section = parser.extract_dag_section(dag_raw)
    dag = parser.parse_dag(dag_section)

    # 기존 방식 주석 처리
    # nodes, edges = parse_block(re.sub(r'^```|```$', '', dag_raw.strip()))
    # dag = build_dag(nodes, edges)

    print("==="*15+" Root Nodes "+"==="*15)
    root_nodes = [node for node in dag.nodes() if dag.in_degree(node) == 0]
    for root in root_nodes:
        node_data = dag.nodes[root]
        print(f"  {root} | {node_data}")

    print("==="*15+" Paths "+"==="*15)
    paths, roots, leaves = get_root_to_leaf_paths(dag)

    for i, path in enumerate(paths):
        print(f"\nPath {i+1}:")
        for j, node in enumerate(path):
            if j < len(path) - 1:
                edge_data = dag.get_edge_data(node, path[j+1])
                relation = edge_data['relation'] if edge_data else ''
                print(f"  {node}")
                print(f"    --[{relation}]-->")
            else:
                print(f"  {node}")

    create_dag_diagram(dag, filename=f'dag_{sha256_hash(msg)}')             
    print()

    

    # break

In [66]:
item_pdf_all.query("item_nm_alias.str.contains('보이스피싱')")

item_id           item_nm item_dmn  \
1665  NA00002114  음성스팸 및 보이스피싱번호차단        P   
1665  NA00002114  음성스팸 및 보이스피싱번호차단        P   
1665  NA00002114  음성스팸 및 보이스피싱번호차단        P   
1665  NA00002114  음성스팸 및 보이스피싱번호차단        P   

                                                                                                                                                                                                                                                                                                                  item_desc  \
1665  음성스팸 및 보이스피싱번호차단&&■  상품정의_  - 한국인터넷진흥원(KISA),SK텔레콤, 수사/금융기관등으로 신고된 번호 및 음성스팸/보이스 피싱번호 및 스팸확률분석 기반,악성앱에 의한_    비정상번호 또는 거짓으로 표시된 번호에 대한 발신/수신 차단 서비스_  -2021.2.22일 부터 기존 음성스팸수신필터링서비스에 차단대상번호 및 발신차단 기능 추가_   ※ 발신차단 기능은 3G 가입자는 미제공되며, 일부 오차단이 있을수 있으며, 확인시 차단해제 조치 가능&&- 별도의 설정없이 가입 후 060으로 수신되는 전화에 대해서 수신이 차단됩니다.   
1665  음성스팸 및 보이스피싱번호차단&&■  상품정의_  - 한국인터넷진흥원(KISA),SK텔레콤, 수사/금융기관등으로 신고된 번호 및 음성스팸/보이스 피싱번호 및 스팸확률분석 기반,악성앱에 의한_    비정상번호 또는 거짓으로 표시된 번호에 대한 발신/수신 차단 서비스_  -2021.2.22일 부터 기존 음성스팸수신필터링서비스에 차단대상번호 및 발신차단 기능 추가_   ※ 발신차단 기능은 3G 가입자는 미제공되며, 일부 오차단이 있을수 있으며, 확인시 차단해제 조치 가능&&- 별도의 설정없이 가입 후 060으로 수신되는 전화에 대해서 수신이 차단됩니다.   
1665  음성스팸 및 보이스피싱번호차단&&■  상품정의_  - 한국인터넷진흥원(KISA),SK텔레콤, 수사/금융기관등으로 신고된 번호 및 음성스팸/보이스 피싱번호 및 스팸확률분석 기반,악성앱에 의한_    비정상번호 또는 거짓으로 표시된 번호에 대한 발신/수신 차단 서비스_  -2021.2.22일 부터 기존 음성스팸수신필터링서비스에 차단대상번호 및 발신차단 기능 추가_   ※ 발신차단 기능은 3G 가입자는 미제공되며, 일부 오차단이 있을수 있으며, 확인시 차단해제 조치 가능&&- 별도의 설정없이 가입 후 060으로 수신되는 전화에 대해서 수신이 차단됩니다.   
1665  음성스팸 및 보이스피싱번호차단&&■  상품정의_  - 한국인터넷진흥원(KISA),SK텔레콤, 수사/금융기관등으로 신고된 번호 및 음성스팸/보이스 피싱번호 및 스팸확률분석 기반,악성앱에 의한_    비정상번호 또는 거짓으로 표시된 번호에 대한 발신/수신 차단 서비스_  -2021.2.22일 부터 기존 음성스팸수신필터링서비스에 차단대상번호 및 발신차단 기능 추가_   ※ 발신차단 기능은 3G 가입자는 미제공되며, 일부 오차단이 있을수 있으며, 확인시 차단해제 조치 가능&&- 별도의 설정없이 가입 후 060으로 수신되는 전화에 대해서 수신이 차단됩니다.   

     item_ctg  item_als item_emb_vec     ofer_cd     oper_dt_hms  \
1665     None       NaN         None  NA00002114  20250101000000   
1665     None       NaN         None  NA00002114  20250101000000   
1665     None       NaN         None  NA00002114  20250101000000   
1665     None       NaN         None  NA00002114  20250101000000   

           item_nm_alias  start_dt  end_dt  rank  
1665    음성스팸 및 보이스피싱번호차단       NaN     NaN   NaN  
1665  음성Spam 및 보이스피싱번호차단       NaN     NaN   NaN  
1665  음성SPAM 및 보이스피싱번호차단       NaN     NaN   NaN  
1665  음성spam 및 보이스피싱번호차단       NaN     NaN   NaN